In [ ]:


import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from collections import defaultdict
from textwrap import wrap

from tqdm import tqdm


%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8

RANDOM_SEED = 23
np.random.seed(RANDOM_SEED)

torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import csv
import pandas as pd
import numpy as np
import os
import re
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds


In [ ]:
data = pd.read_csv('/content/trac2_iben_train.csv')
data

,ID,Text,Sub-task A,Sub-task B
0,C45.688,Dada taratari,NAG,NGEN
1,C45.635.5,Tumi korbe Amar sathe,NAG,NGEN
2,C47.103,Ar ta chara a sob bessha peter cheleder okhan ...,OAG,GEN
3,C68.147,কাকের শরীরে ময়ুরের পাখা লাগিয়েছে,CAG,NGEN
4,C45.615,পতিতাদের চরিত্র রাজনৈতিক নেতাদের থেকে হাজার গু...,NAG,NGEN
...,...,...,...,...
3821,C44.44,বালের সব গল্প গুলো,NAG,NGEN
3822,C68.420,Khub vhalo lageche video ta.... Ranu k Aro Aro...,CAG,NGEN
3823,C68.1277,abar 4 no.platforme jabe bati hate niye,CAG,NGEN
3824,C59.1122,Ghinna mondol,NAG,NGEN


In [ ]:
#data.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
NAG_train = data[(data['Sub-task A']=='NAG')]
OAG_train = data[(data['Sub-task A']=='CAG')]
CAG_train = data[(data['Sub-task A']=='OAG')]

In [ ]:
import warnings as wrn
wrn.filterwarnings('ignore')

NAG_train["Sub-task A"] = 0 
#negatives_test["Sentiment"] = 0

OAG_train["Sub-task A"] = 1
#positives_test["Sentiment"] = 2

CAG_train["Sub-task A"] = 2
#neutrals_test["Sentiment"] = 1

NAG_train.head()

,ID,Text,Sub-task A,Sub-task B
0,C45.688,Dada taratari,0,NGEN
1,C45.635.5,Tumi korbe Amar sathe,0,NGEN
4,C45.615,পতিতাদের চরিত্র রাজনৈতিক নেতাদের থেকে হাজার গু...,0,NGEN
5,C68.303,Thik bolecho bhai,0,NGEN
7,C44.146,অমানুষ,0,NGEN


In [ ]:
NAG= NAG_train['Text'].tolist()

In [ ]:
OAG= OAG_train['Text'].tolist()

In [ ]:
CAG= CAG_train['Text'].tolist()

In [ ]:
CAG

['Ar ta chara a sob bessha peter cheleder okhan diye dhukano dorkar',
 'এই মহিলা মানসিক রোগী',
 'CAMERA KORTE SEKH BOKACHODA\nTARPOR CHULKATE JABI',
 'beti ta khub soytan bikari . manus re khub opman kore.',
 '@FunMasti Tv are bokachoda cam ar jonno nittisto akta Manus thakbe taibole\nbonke noy janoyarbacha Tora to maa bon ke chudis Tora ai pobitro somporko ki\nbujbi',
 'Upni bodhoy jan nijer jonno kotha sune tai to mone hochhe. Upner moto muske ei\nvideo korar jonno to juto peta kora uchid. Karon upni abar oder choto kore\nnije imcum korte chaichen',
 'Kala magi',
 'বাস নিয়ে ভোরে দিতে হবে শালী কে',
 'Khankir may amn may k puria Mara dorkar',
 'Odar ka bichi kata dia dorkar sala harami dar',
 'Ranu tho eakta bal chira singar.',
 'তোমার চামরা গোটানোর দরকার।\nশালা ভিডিও করো।\nতোর মার মতোই রে ওই মহিলা।',
 'Puriye mara darkar',
 'ভিখারী ও ছোট মনের মেয়ে',
 'Bharot rapist der desh hawe gache.. Je deshe Ma meyeder kono samman nei sei\ndesher konodin kichu habe na. Obhisap jabe kothay. Fucking

In [ ]:
data= pd.concat([NAG_train,
                OAG_train,
                CAG_train
                 ])

In [ ]:
data 

,ID,Text,Sub-task A,Sub-task B
0,C45.688,Dada taratari,0,NGEN
1,C45.635.5,Tumi korbe Amar sathe,0,NGEN
4,C45.615,পতিতাদের চরিত্র রাজনৈতিক নেতাদের থেকে হাজার গু...,0,NGEN
5,C68.303,Thik bolecho bhai,0,NGEN
7,C44.146,অমানুষ,0,NGEN
...,...,...,...,...
3795,C59.2318,Ora gushti sudho pagol,2,NGEN
3798,C44.336.1,এরা সব এক একটা বেটিচোদ।\nএদের জন্য সমাজ নষ্ট হ...,2,GEN
3808,C61.1815,মাদার চোদ রানু,2,GEN
3814,C44.226.4,@হিয়াশা জামান হীর orai amr maal khaa. Tate To...,2,NGEN


In [ ]:
data.drop(['ID','Sub-task B'], axis=1, inplace=True)

In [ ]:
NAGhin = NAG_train['Text'].tolist()

In [ ]:
#data.drop(['Unnamed: 0','index'], axis=1, inplace=True)

In [ ]:
data1 = pd.read_csv('/content/cagbeneng.csv')
data2 = pd.read_csv('/content/oagbeneng.csv')
data3 = pd.read_csv('/content/hinengCag.csv')
data4 = pd.read_csv('/content/hinengOag.csv')
#data5 = pd.read_csv('/content/train.csv')


In [ ]:
data1

,Unnamed: 0,Text
0,0,"We dont want to be caste-bound, we used to kil..."
1,1,@ Feminism is Cancer * Un Feminist Yeh Sahi Ha...
2,2,Ilia Jat Jhant Garam Hai Mukhya Jat Hu U Nehi ...
3,3,Lit Sirvans Des Bhakti Mai Bhai Item Geet Pelege
4,4,"We need men in the army, not eunuchs.Indian so..."
...,...,...
516,516,"@ Itch Pong Abe Randi ki Aulad, what has happe..."
517,517,Arundhati your janma diya maap gyan ji hi to m...
518,518,mutiny
519,519,Kung Thu Gadhiya Hai


In [ ]:
data1.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
data2.drop(['Unnamed: 0'], axis=1, inplace=True)
data3.drop(['Unnamed: 0'], axis=1, inplace=True)
data4.drop(['Unnamed: 0'], axis=1, inplace=True)


In [ ]:
data2

,Text
0,"@ Sucharita Tyagi Jhuthiya,"
1,"Liberal too, tolerant too !!! Hilarious laugh"
2,"What is there now on the government, not Bhosr..."
3,"Kya yaar review leah lee lee ko read a little,..."
4,"So what are you girls, no Mansi"
...,...
387,Bhai Lakhai Chaudharys video is Hatwa Diya. \ ...
388,Manna Awaaz Aachi Woh Lankina Kina Din Raghagi
389,Acha to tune is film had to be seen with dekay...
390,Kabir Singhs Ekke Hamad Bad article was releas...


In [ ]:
data4

,Text,label
0,There is also liberal there is also tolerant L...,1
1,"Does no one trust the government anymore, what...",1
2,"What a friend to review, write down, read a li...",1
3,"This happened in the right justice, which B*%$...",1
4,So shouldnt there be girls too,1
...,...,...
700,"We are all equal, talk u. Big countries like S...",1
701,Rahul Kumar Gandu This is not a law in Pakista...,1
702,"Well then, our population has increased from t...",1
703,Hear his answerYou redeemed it Pratik Bhai.......,1


In [ ]:
datas= pd.concat([data,
                  data1,
                  data2,
                 data3,
                 data4
                 ])

In [ ]:
datas

,Text,label
0,Next part,0
1,Iii8mllllllm\nMdxfvb8o90lplppi0005,0
2,🤣🤣😂😂🤣🤣🤣😂osm vedio ....keep it up...make more v...,0
3,What the fuck was this? I respect shwetabh and...,0
4,Concerned authorities should bring arundathi R...,0
...,...,...
700,"We are all equal, talk u. Big countries like S...",1
701,Rahul Kumar Gandu This is not a law in Pakista...,1
702,"Well then, our population has increased from t...",1
703,Hear his answerYou redeemed it Pratik Bhai.......,1


In [ ]:
#data.drop(['ID','Sub-task B'], axis=1, inplace=True)

In [ ]:
print(len(NAGben))

2078


In [ ]:

NAGBENSTORE=[]
c=1750
for i in range(50):

  OAGBENSTOR = NAGhin[c]
  NAGBENSTORE.append(OAGBENSTOR)
  c=c+1


In [ ]:
print(len(fNAG))

1781


In [ ]:
NAGBENSTORE

NameError: ignored

In [ ]:
fNAG=[
      'Bom Bom Bajega Bom Bom',
 'You will not know when a psycho stalks your sister or daughter',
 'The message was very good but the laughter came...',
 'Ranga - the leaders of BJP who survived the rapist KO people. \nBadla- 7RCR Address of our Prime Ministers Office...'
 'Partik Bhai\nIf he sees Ramayana by mistake,\nthen he will say that Lakshman did wrong with Supanekha',
 'Bhai Bhaut time later the street listened \nmensutra is back',
 'live good video good video',
 'AP. Battling like Babaje,,, 🤣😂😄😃😁',
 'Great brother'
 'A movie is made on Sir Falls 498A or CrPC125 also Koi Himmatwala',
 'Excellent...'
 'Final Cut News My Dikha Raha Hai has been raped...! Lets elaborate\nbrother',
 'Brother, just like you and made a long video too crow detailed me..zur\ndekhenge🔥😘',
 'Brother we have to stop being influenced by movies. We have grown up',
 'Liberals are no longer even worth discussing',
 'Whoever does not do justice, he should remember this matter',
 '[13:23](https://www.youtube.com/watch?v=N_ZMfQMZos0&t=13m23s) \nBhai Bhai.....Bikul Sach Bola Re',
 'Brother you make a lot of content, yet subscriber is not bad... you eat last me like, comment, subscribe... such b\nrha, it is a very different post.....',
 '@S Kumar Haji',
 'I have spoken right now sir...I am in 12th class now...but my life and career is just ending and the reason is the same',
 'The Godfather, Inception, Got Review When Kroeg... Reply',
 '@nripen rajbongshi Hy',
 'Is the bar good or has the film come on different subjects like section 375,\ndream girl, wrestler are all good stories',
 'What to silence Kabir Singhs logo',
 "I, S, Jasha, Karega, Oyesha, Varega",
 'Same HI HE... \nJust 2 different names',
 'Brother please review the fault in our stars',
 'Ek number video sir, fun aa gaya',
 'Baby Main Nahi...Bhaiya...Akshay Kumar..Itam Geet',
 '@VivekShahi Namaste Uncle Moody Sahab has how many fathers. Now thik j bhai?\nWrote politely...',
 'Stop this by subscribing to...',
 'What is the matter brother',
 'Something you say girl Kamal is... Hot',
 'Hello rehab sir,\n it is pleasantly pleasant. \nwe'
 'Akshay Khanna is a great actor friend',
 'I was remembering Swara Bhaskar every time I heard the word liberal Thedosto\ntmhe who remembered the comment crow.',
 'Real Critics Watch MovieStock Review Of Bhai Jaa',
 'Brother, your book will return a thousand love letters whether not on Flipkart or Amazon\ntried not getting it',
 'Man.. here comes the ghost taras.. so bitter.',
 "Kutte Ke Tara Loves",
 'Watch full video',
 'Sir, how or from where do you get to know what\nthe Akalists or Liberals say?',
 'Bollywood should move a little beyond love',
 'Mumbai chakla my number is 8286209041 contact for body massage',
 '@Akash Kale Absolutely, the rest of Chinal Os negative reviews are also starting to come\njayega',
 'How many gimmicks do you do brother, you complete Delhi',
 'Given a good explanation brother, I also did not like the film',
 'Its anything!',
 'When the theme of the movie is political H2 review will also be in the same way',
 'Dont change the name of the channel brother, there is confusion'
 'Faltu film ha ya,,, doctor aisa nahi hota ....... r medical college aisa\nhota to ...... led, AIIMS is not so difficult.....',
 'Naked em of mob lynching keep you poor',
 'Nupur Sharma',
 'What btc🤣 \nShuru k 10 sec',
 'Bhai Meeto Tera Phan Hoon',
 'Vikas Mishra you are right',
 'Brother app said very nice...perfect...',
 'Wheres your right...',
 'Sir so late video......',
 'Thank you sir for this video..I havent seen Kabir Singh..but really its a \chutiya film',
 'Those who are destroying the society have talked about them... but what to do\nthe truth is bitter, so maybe the truth is not being heard from you!',
 'anything',
 'Bakshdar',
 'Vinay Daharwal to jeans pant chhod then',
 '@onkarnaik9250295775',
 'Everything is fine..control the speed of speaking a little',
 'Brothers words come true',
 'Chutiya is with paan',
 'thank you brother',
 'Kya yaar reviewing 3 great old movies',
 'These people will agree if they see this video of the film nor the app.',
 'Rights of every army person and civilian would be different',
 'Reviewed very quickly',
 '@ Thugs of Hindostan',
 'Please make review of KGF movie',
 'Lets say something right brother',
 'Organization of family members. Without the crime\narrest was not unconscious.',
 'Ha ha badiya bhai 16 na true heart happy brother real said brother',
 'Where are you meeting so many liberals',
 'Brother when will you be on Instagram???',
 'This movie is really a mess',
 '@AmreshSahai you know something big except LGBT',
 'Shweta Mehta You didnt even tell me what would you do if you were a lesbian or a lesbian\n.',
 'Bande Ki Jamaat Hui Kya. PLZ Update [SOMEONE.IS](http://someone.is/) He\nRelease or is he still in jail.? The Jamaat will give everything as much money as it takes.'
 'should be yes',
 'Modis son',
 'Awesome Movie Shahid Always Rock Shahids Blockbuster Movie Got Burst\n Achhe Achhe Ko Shahid Hai Rok',
 'Radhe Krishna will be your mother too',
 'Brother, you can mount such humiliation....of the movie',
 'Right brother'
 'E to Z, no matter what everyone is entitled to for our country everywhere',
 'Abhishek Kumar Mast Bani J',
 'Arundhati Roy and Sirdesai collide',
 '[2:10](https://www.youtube.com/watch?v=ZzsAuDkXq1M&t=2m10s) If our child is\ngay\nthen we are',
 'Wish dowry and give punishment.'
 'Some friends are left in the cell, yet have to be kept in life',
 'Where was the brother review',
 'You did it right, but how did you give a thief to your wife',
 'First of all you have to clear the concept of your left-right conservative-liberal\nchahiye.... \nWatch this video - \n<https://youtu.be/yNdB772uAsU>',
 'You talk in Hindi this is the best part of your channel and you\nunderstand good buyers'
 'Its not about you and me... Its about those boys who will commit crime by taking this \ninspiration.',
 'The song Pratik Bhai "Urvashi" was shot for Kabir Singh but now the song\nwhy didnt come in the movie.',
 'Pratik sir you make a long video mount \nnot interested in watching dough',
 '@AakashKale Is it so easy to do drugs and alcohol in real life?? \nIn the movie, he leaves everything very easily... \nIf he had finished, it would have justified everything',
 'Just you peoples support, tea will give you the fruit of our mehndi 🙏',
 'Janata - Modi ji GDP has fallen\nModi - with friends he himself Vaibhav ji\nBhakt - Modi wow what would you have heard',
 'Looks like I dont have the money to watch a movie in The Hall... When the film was released on TV\npe, then I have come to analyze it..',
 'What is acting, Chandu really got a big role, that day was a big hit. \nJanidar 2001, Chandu So',
 'I want to do friendship with me, call me phone rate, where is my number 6393830912',
 'You become a hero mount by reading this meme',
 'Love like a dog is right',
 'Jio Vais',
 'You have won a strange heart',
 'What kind of love would have happened to brother Dara',
 'Oh brother got TRP...',
 'Speak right, but this too was dumbo in the film, it absolutely means doing yourself\nwhatever he does or not, poor Obir should do',
 'Brother talk in Hindi',
 '@AliKhan ',
 '498 as false hair business is not a block'
 'Napur',
 'Sonic Raj Bhai Mako Bi Been Watching Bollywood Movies Over 10 Years',
 'Sir BS bo apna kimi shi kray baaki koi what should we do in our personal life\nShould MTLB have so much freedom so that bo can purify his dreams\no matter what field..',
 'Shweta Bhais respect has increased since this video...\n\nPeople of Varna are gathering ideas to sing Kabir Singhs song Aaj Tak...\n\nLap Captain I have seen only some holidays. ..',
 'Watching the premiere show',
 'Brother I was waiting ',
 'Bhai Bhot you have given a great critically review!! Best Speech - "Pant Mein\nBarf Dal Liya" ',
 'He said towards you sir... had you to be a director',
 'All right brother'
 '@nripen rajbongshi 19',
 'Very good brother'
 'Gentleman 123450 you dont seem to understand my comment, brother, I am also telling what you want to say, I gave an example.',
 'Arundhati Roy has suggested this, not right vais ki aruthi her\ oy bar apna jais re nu rang billi and get your name wais ris course road rode, if that magic is again powerful then\nfollow the idea',
 'Dear post brother. Or I wont say anything',
 'Brother Rose Video Upload Do Do Video',
 'Spoilers warning cone',
 'I have not seen this film but this kind of film should be made because\n action is too much for you, its fun to see\nI am still 5 years old and not as many Bollywood films as Mehndi kione even Must have seen 4-5 movies of count',
 'Madam app first find Bengali !!😂😂',
 'He was faithful at work',
 'I dont have faith in the government',
 '1 no film j bhai',
 'Tho Pakistani people said Bodo Indian Army',
 'No work is needed in the army of filth',
 'Absolutely right, good review',
 'Have to keep a dictionary while watching your video',
 'Its your decision.. first like then watch the video',
 'Kon koan hai jahan kabir singh (shit) still not seen...\nhit like...',
 '@VijaySharma read brother read \nread carefully \nI have read this, you also read a little and read...',
 'Whats wrong with the girl, brother? \nSet aside sexual feelings for 2 minutes. \nThat person is that \nNot just those 10 days (bath) you yourself will run away. \nIn Bollywood, sometimes you fuckin heroine Sandas caste that barber',
 'By the way, by watching Prateek Bhaiya AP to four more shots please, you too had become\n🤣🤣😂😂😂'
  'You are the right thing man.. love uh brother',
 'The dog was named Preeti, making a ruckus for that too, on Twitter',
 'Answer in English',
 'This guy has done the right thing!!!!!',
 'Dont think of the boys suffering from the false dowry case of Ladies, teach a lesson to the family of Jahangirs\n such ladies',
 'What is the problem of ghost families with the act mh please change the rules'
 'Homosexual ke bare mein hum kuch special ni pata j ya jeeta vee dekhte ja woh reality\nshow mai dekh je which will be able to fulfill the needs of KV V Army..',
 'Sir please provide the magazine of December...',
 'Listen to what I said in the ending of the video... Wrong so many sarees\nBate is happening...\nIn our country, criminals and terrorists are glorified\n in every second and third film... What can do?',
 'Brother honestly, have liked your video, app also achche dik rahe ho from naturel lights',
 '[18:58](https://www.youtube.com/watch?v=N_ZMfQMZos0&t=18m58s)\\----Your brothers\nbehn chodta rehta hai😊😊😊😊really????',
 'Dil Se Boss',
 'Spoiler warning to be given',
 'People like you can support the system of power',
 'Once my father had new food, I would kill him... I heard that one sweet chin\nmusic...',
 'Are you a friend',
 'So many subscribers in a channel with such a crappy critic?? Hey, God ! Good talent\nSubscribers are dusty like Dino James, Salil Jamdar etc...',
 'Auto Wale Bhaiya :)',
 'Rachna Mishra was making a ruckus So I am sure I should do in Karva Mishra? his n,',
 'Sir,\nGaurav sirs current affairs are not coming on daily basis???',
 'The ghost of subtext is used well then in the film',
 'T-shirt is great brother',
 'T-Shirts will change well',
 'Today you have answered the hypocrites. Well done ',
 'Told the whole story by telling your story',
 'Kya aap ki left cheek mein dari yada hai, this is my phone screen bad!'
  'After watching Chutiya film Tha Yaar Maine, Laga Ke Yaar Ki Dekhliya Maine.'
 'Is the film a sex drug or a drink hit, the same Sanjus tragedy...',
 'Great movie'
 'Kyo hai who doesnt see this crap movie like me..\nWho already knew this is going to be rubbish'
 'Pratik Ji Ram Ram',
 'Sir ji please support our kevari river through kevari calling\nwhich is also supported by Prakash Javadekar ji himself',
 'Lazbab',
 'My life is just for your arrangements.'
 '@NikunjSingh Oh Acha',
 'বা: মেরেসের া:',
 'Wanted to see your review of Kabir Singh from Bhut Din,'
 'I really liked Kabir Singh',
 'Did you see it uprooted??',
 'From Banana to Banana...',
 'Tell Nitishba how the paper was leaked',
 'Right brother'
 'True Brother'
 'Sir [real.love] (http://real.love/) nhi smjynge ....y log ..film ..film\nto.hansu h .esa man to.ajkl.nhi h',
 'Yeh Londa a little more hoarah that..',
 '@SnehaSharma Yes, I remember the video, but dont know their names',
 'Suryavansham',
 'Hi how are you',
 'Brother brother brother brother',
 'Video n message was good but girl. Didnt like the sound of the voice of the sound of hi down tha\nbold 1...... otherwise its good 1'
 '@max Monika you make your opinion by watching such fake and baseless videos \nPlease dont make yourself a laughing stock... \nIf you want to live naked in Hindu civilization then watch this video -\n<https:/ /youtu .be/_5dJyQdBsD4>',
 '@shashi karne yah bhaat he LGBT ki kar rage hain',
 'Its right'
 'Were doing good self harm... why didnt you say earlier....🤣🤣🤣',
 'Right where man',
 'You are right sir'
 'Serius Na Le Woh Sirf Ek Movie Thi... Sanju Movie Sabse Chutiya Aaj Tak Ki',
 'I really liked the total scene of they walk free in the film.',
 'Thume Ka',
 'Brother, there is no doubt about the film Tu Achhe, but the message I had in the film was not realistic, so it is stupid to compare it with real life',
 'This T-shirt is as old as one has to say',
 '@arjun bhabad 67 percent have not chosen Feku.',
 'Laundiya*',
 'Vipan Singh Rajput Legal Force',
 'Best Bollywood movie ever... Movie Dekhte Baht Achcha Laga.. Asli Kahani\n👌👌👌',
 '💯% best best ️❤️❤️❤️❤️',
 'Watched the movie today. Really nothing is worth watching except Shahid Kapoors acting in the film.',
 'Brother, whatever your review is, I dont pay attention to it.. I watch every movie\nevery film looks good. Just the point of view is right\nchahiye..but i watch your video only because\nI like you a lot. Saying words clearly and agreement is a big\n thing...',
 'Sir Pichle no video was coming since 1 month\nnow released since days\nThanks',
 'Make a video on what 20 year old London should do at this age......\nPlease make a lot of videos from them but brother come from you.. please make this video',
 'Brother they should watch RX100 movie, Souths TB will know',
 'We cried for the last 7 minutes of the film, I could not control myself...',
 "love like a dog",
 'Who shook seeing the one above..?? \n🤩🤩🤩',
 'No brother, you didnt even understand the film. Remember that Nirbhaya scene. The minor word\n is released. (Strictly because of the law). And in the last, when\njudges give the decision, they clearly do it according to the law, mentioning that they are getting the advantage of their\nstatus, then that too would be rape, hi thought, even if\nthere is agreement, why not.',
 "My wife was Lei" 
 'Good brother'
 'What are you talking about',
 'Bhi Jaan Tik Tok bare me message',
 'Love you boys!!!',
 'Bhai was not an item song in the film.',
 'Reality Me Kabir never finds love, what should he do man',
 'I should be sent back to where I was',
 'Sir plz 1 vdo 2018 imp award winners pr bnaiye plz...',
 'Eagerly waiting for this',
 'Its all fine too, this launde watched the movie today\nIslia is reviewing today\'s download',
 'It was a very dirty review',
 'Arundhati Roy Zindabad',
 '**Namaskar Award Sir**\n\n**10 World Hindi Day**.',
 'Farji movie was...',
 'Brother you are spoiler alert to date',
 '@mohanish kinare Hanna',
 '**Kon wants Pratik Bhai to join Battoon hata de chenal se**',
 'No..if Indian is to be served...or even trike...',
 'Forgot to shave one side'
 'Brother, you have explained in the right way, love your review',
 '@shivam kavitkar it is better if such people are allowed to go',
 'If you come from Bollywood, you have to go and watch the whole community film, you have to stop\ndega..bs to speak the truth like this, brother ji, jiski bhat need h young people..' 
  'How many people have not even seen Kabir Singh movie like me',
 'Brother give reaction on family man',
 'Its good brother, you didnt see his name or photo, the character like the strike on the target \ nchoudhary comes on you too',
 '[3:26](https://www.youtube.com/watch?v=N_ZMfQMZos0&t=3m26s) like a cut\npyar kriti/ krita hai wtf bro🤣',
 'Baskhes Mast Kar',
 'We should learn to speak truth from PM',
 'What is the name of your place',
 'We had to see Mission Mangal but all the friends were going to see Kabir Singh\nthe... or when we said lets go to see Mission Mangal, everyone started laughing or said \Cutiya ho... Thats why we had to see Kabir Singh under compulsion. ..then war\nor Joker I had to see Joker but people went to see the war.',
 'Sir woman has accepted friendship till now',
 'Memorasu Nothing after having the wrong case. Wrong is wrong\ns. I dont even protect men. So what does he do? Only'
 'Make a video on Kabir Singh Vs Arjun Reddy and yes you said anything about the actor\ns acting in this video no pls make a video on it sir ji',
 'I have got to learn something in South India movies.. I have to watch Bollywood',
 'Me Ne We Mardi',
 'If some of us abolished the SC, ST, OBC reservation, then reserved the economically backward people.',
 'Right Shagun Ji Best Analysis Prateek Sir',
 'Zbardest Video Bhaiyo, Carry On',
 'Bhaiyeeeeeee dil luout liya aapne shukkar hai people like you\nabhi hai in india whose thinking is the same as my thinking\nhaieejeeeeeeeeyoooooo bhaaaiiiiii bass you',
 'The movie is very good .... everyone should watch the pain material p movie km only it is made',
 'Kabir Singh forced ASI film Jisko Swetaab to watch the film, brother did not eat\nt watch the film',
 'You are really unbelievable brother...\nso many rats you shoot videos too...aside from movie review, pay attention to your health\ngood review....\ngood midnight.. .',
 'have you gone mad? Who would want to have sex in the open? I would never.',
 'Ill go for a walk'
 'AJ Main Kabir Singh Dakhi... Darun Lago...on Netflix,'
 'Why cant I kill a girl, I can kill a boy If you really love'
 'Brother, petrol was better than all these',
 'Ranga Billa is Amit Shah and Modi. .',
 'But some people of the house are giving shelter to the intruder... but the watchman\nwill take everyones life to drive out the intruder in the house\nhi...the house is not full of just one man... .and here So\nthe whole joint family is that of 130 crore people... why didnt you shake it...',
 'Brothers bring the issue... Vote only for Modi ji Bass and Nationalism\nko follow karo !!',
 'This film has shown reality. So whats the matter?',
 'Bhai KY Bol Re Tu..',
 ' @Chirag First of all you should clear your Left-Right Conservative-Liberal concept\n... \nWatch this video - \n<https://youtu.be/yNdB772uAsU>\nAnd see what is going on in the world ... leave theory practical\nthink...\nI myself live in a village which is not even a taluka... and I am sure that I have seen more poor than you...',
 'Two films were also of the same character, neither should the director be the same',
 'Can be served',
 'prashant sir sbse good smjhate h',
 'Super Super Boss Jio Boss',
 'Very right brother'
 'Brother people swear this diwali no movie..only spend time with friends and lovely parents..and crackers (Ayurvedic crackers) powered by Nick from "Priyankas Herbals" and crackers (Ayurvedic crackers)',
 'Great brother'
 'This is a very nice movie...Watched today 5/5 rating is mine',
 'It is not a mistake, but even doing a show is okay',
 'The shirt is awesome',
 'He was not in the mind of Lakhi.',
 '**Kabir Singh** there was no time to see the Essie picture... **When did I know when\ngayii😁 did not know ** \n **Seriously apun ni did not see**',
 'Kabirs whip disinfected',
 'I watch the trailer of the year, he went to the society that this is also the same crap film\nBollywood',
 ' A soldier of mine wants a soldier and not a man, or a woman.'
 'Rahul Pandey is right',
 'Brother loud',
 'Why anime is not seen in India, that too in Hindi and why is it a useless cartoon\ndikhate, make a video on it'
      'Brother, you have won the heart, have you given the answer',
 'What did you say brother Bhot hard !!!',
 'Hindi X',
 'No homa should army me',
 'To be Conservative and Liberal from Indias point of view\nWatch this video completely different - \n<https://youtu.be/yNdB772uAsU>',
 'Last me she used to say that she had joined the clinic etc',
 'Today Saurabh Trivedi hacked Rajdeep Sardesai',
 'Bancho movie unrealistic and I am benchod first year commerce student after watching movie\ninspired for MBBS\nthinking to take NEET exam with +2 science,\nthinking of taking NEET exam',
 'Jai Shri Ram\nVande Mataram',
 'Jio Jio Boss',
 '@Guderian Tanks right',
 'To make sure',
 'Girls are easy to earn money',
 'True brother..',
 'First 3 minutes Ive seen your videos. I have seen the movie or the vi review of the logo. But no one tells you apart from your star review.',
 '@PratikBorade I had seen the full video. I wasnt talking about you,\npeople in the comments section were saying. And yes I agree, if you dont think bare of all these\nthings, then Kabir Singh is a very well written and directed\nmovie',
 'After listening to the review, it seems that how will the film with such a careless be seen once\n??? :D',
 'On 23 May 2019 the Public of India has rejected O Liberal',
 'Pragya Thakur sings. Nathuram Godse. Were all these also Muslims.??'
 '@ Hassan Sardar \nWhat happened?😃',
 'Cansa address plz and number do',
 'If I say one thing I wont mind',
 'Brother, this video is going on a different line, brother',
 'A girls pitch is 100 boys but Omar thinks it right,Think h career between numbers 15-18, Dayan Dina Chiye,'
 'Jio Guru',
 'Jio wai very good.',
 'Abhishek Palhi',
 '@Ching pong koi na mere brother.. in your age we too are like yours... now we accept virtual\nreality as truth... but the truth is bigger than time',
 'Crap picture.',
 'Nothing good comes from Bollywood',
 'Right said Sandeep Bhai',
 'Hey sal bhot was good vado and comic too... Manjeets acting is very funny\nhis expression ',
 'Bawal bhaisaheb.. you said very right',
 'And Mr. Berozgar bring the Purano news',
 'Everyone wants a feudal right,'
 'Sweet ',
 'Good man. we people. hi cathay. so be it',
 'I havent seen a single Bolishit Movies from bc 2-3 years ',
 'Say straight in whose naked you are speaking',
 'I laughed a lot [17:26](https://www.youtube.com/watch?v=N_ZMfQMZos0&t=17m26s)\n... \nkabir : I took the injection. \nPreeti : [17:26](https://www.youtube.com/watch?v=N_ZMfQMZos0&t=17m26s) Hey\nWhy didnt you tell Bhosadis',
 '@AhmedBaloch Bhai I can share your pain,...\nNo problem, it will take some time to heal the burn, but it will surely be filled. \nbe patient',
 'I will subscribe'
 'Absolutely',
 'Some people will say... to play peoples work',
 'Kind mercy',
 'Jai Shri Ram.🕉️🕉️🕉️🕉️🙏🙏🙏',
 '😂😂😂😂🤣🤣 brother wow what a video you have made….. made me cry. ........ after section 377\n......😢😢😢😢',
 'Kutte ki Tara is loving- the best adjective...'
  'Brother plz make a video on this.....<https://youtu.be/Wh1583naLkQ>',
 'O Bhai Point Pe Aana Bhai Kya Kehna Hai',
 'look to review',
 'I have lost my brother... Lamao',
 '@jj the mind doesnt get spoiled while watching a movie with politics',
 'I dont know how she listens to Ohkar in Ranos talk, but I didnt like the sound of Rano jis gala.',
 'Wow shera son, then its like this',
 'Bhai Punjabi Movie Review Crow',
 'Sunil Barmade',
 'Awesome video',
 'Jai Shri Ram\nVande Mataram',
 '@BharatBehera Facebook ID Do Apne',
 ' @PratikBorade saw the movie brother. We also understand right and wrong. n ur\n point of view too.',
 'Whoever made me dislike Kabir Singhs evil cream more than him, whether it is\nRajeev or someone elsearjun Raddi n Kabir Singh donny master piece that no doubt',
 'What was the matter of apologizing to the brother, who would have shot the video after taking off the glasses',
 'Lande',
 'It is being remade in another Tamil language in South',
 'Brother your reviews are always perfect when I saw the movie very good Shahid Kapoor did a fadu acting',
 'Very well'
 'Hello Hero Ji',
 'Gehind',
 'Lajbab Film',
 'O woman will you befriend - ha ha not with this man.',
 'Sir Please Titanic Movie And Back To The Future Review Kiji Please Request',
 'Tell me clearly what you want to say.. dont confuse',
 'We want vedios on CBI.........',
 'Sir, how to choose a she life partner, what should be kept in mind\n made a video on this for boys..',
 'Sir Tik Tok PR Video Banana N',
 'You are my teacher',
 'Its actually the opposite',
 'There is no investigation, such fear is taken',
 'Brother, Ramadhir Singh had said this a lot already...\nAs long as there is Bollywood in India, only sap chutiyas will continue to be made...',
 'Brother, enjoyed seeing your review\nNice Explain Keep It',
 'I got a lot of relief :)',
 '@Master Devoe hamesha ladke galat nahi hote wala video me.',
 'Shaa baat hai girl 498a felt that Jootha or the whole family gets to do more \\ I dont bother at the time of Nage',
 'There is also a problem with fixing the problem this way'
 'Ana jaroor in saddi... I have a thought in the naked.',
 'Mat the stereotype. The LGBTQ movement opposes exactly that. Video\n How many Uncle Aunty supported.',
 'Stree me friendship means something else, symbol brother',
 'Dont advertise so much..Sir..Vado look..dont like it..interest and\ncontraction goes away..this is what Sandeep Maheshwari told',
 'Lajobab',
 '@AlijahHusseinMirza?',
 'Let lust and sensuality have a life that is.',
 'Brother, it seems that I too have started thinking of you because I had the same\n same opinion...',
 'Where did the elephant king go',
 'When will Shwetabh sir please analyze Laal Kaptaan movie????.....,',
 'I hope you film bring some change in peoples mind',
 'Indian constitution we log to Soche kisse se bhed bhava Nahi kar\nsakte par practically possible so Nahi Hoga ki samaaj ki vichardhara ki\nbadal Paana sambhava aasan Nahi according to ke. I see you all equal but all such \ nsochge is difficult.',
 '@Arjun Kalsai Pt. dont give'
  'Pratik Bhai has given 10/10 stars to a film for the first time. Then it is made to see this film\n.',
 'Once upon a time a soldier was found in the train after me in the night to die,'
 'As long as there is cinema in the country, people will keep making jokes Ramadhir \nSingh',
 'What are you talking about @vaty Thugs Of Hindostan?',
 'Review Joker Please',
 '100% correct!',
 'What is special about the channel is that it is open to many types of views',
 'Oh lady will you frisip',
 'Army, I want to choose such people who focus on your work and serve your country. Are they gay or someone else, there is no difference with the bat\nhi pdta.',
 'Think of thoughts Babu Bhaiya',
 'If the world is to be shaken...💪💪💪\nYou must stop moving yourself...',
 'Must be gay sir india me army because focus on their work\nchahiye person al life not If their no wrong than a homosexual h\ni natural h. Those B people look for service. \nAnd you should give Ken the face of your proven talent.',
 'Yeh Laddu Roy Kon J',
 'Brother please... dont make fun of holy words like Jai Shri Ram man\nplzzz... meaning is it necessary to say Jai Shri Ram at the end of every video\nkya...its like a mantra. ...plzz',
 'Brother 2019 I will vote for Modi and AP? Har Har Modi Ghar Ghar Modi',
 'Somebody leave this railway station please',
 'Never be sure that it will be sure forever, so be sure that it will be sure forever.'
 '@seemanchal mishra rightly said',
 'I have gone to the film on your own advice... it is a very good film',
 'Big author!!!!!!',
 'Neha Gupta is saying that .. the teeth were taken out due to the lions fascination ... wah\nwaah .. means the woman is a lion .. in the eyes of the laws',
 'Where did the hair come from, Re Baba',
 'Gf ki shadi ho rahu hai brother my dirt is torn please make a vedio\n over it',
 'Bhai watch movies for entertainment\nnot for motivation\nwatch tera videos for motivation\nSo har gaal mein bachhodi mat pelo',
 'When will a woman be friends?',
 'Hail India.',
 '@MarySharma, Madam, I even eat that "not in a public way"\nwont do it in public as we do in private\nbut now they will accept their relationship in front of everyone. Having teeth.',
 'Sir then Sabi should not be labeled as Sanghi',
 'Salute this man the porcupine had presented to be presented\nshould have donated blood of blood of blood of blood',
 'Change your mind again',
 'When will the next season of Bhai Umbrella Academy come?',
 'Brother was drinking water ..but suddenly\n[13:08](https://www.youtube.com/watch?v=N_ZMfQMZos0&t=13m08s) sunkr...friend k\nmuh pr pichkari pad gone ',
 'Who is this liberal and feminist..?',
 'What happened brother',
 '[0:04](https://www.youtube.com/watch?v=N_ZMfQMZos0&t=0m04s) Gangs Of Wasseypur\nDekh Ke Aaya Ja Bhai :D',
 'Yeah....very big movie H',
 'Somebody on YouTube is the one who sings Kabirs nonsense message, compromise or split\n',
  'Bhaiyo, this video should reach Bollywood or eyes should be opened',
 'Right brother',
 'I was thinking when will I give a review of Apna Bhai Hai movie, brother, my review was also the same after watching the film',
 'O woman came tomorrow \n \n \n \n \n 12 oclock yesterday',
 'Sahe said brother',
 'Look at this, what is the compromise problem in the film. \n<https://youtu.be/4BPU792CdM8>',
 'Take a review of the rest of the film too early Lali',
 'Bollywood Gurbez as soon as he heard the video...👍🏽',
 'should be...',
 'Everyone should have equal opportunity',
 'And the movie Khamoshi which was released recently.',
 '@Precious Paws is right, its hair will be thinking.',
 '@alijah hussain mirza VIGO VIDEO JOIN KIYA KI NHI DEAR ... PLZ JOIN KRLO OR\nKRE MOHDSAIF SEARCH OR FOLLOW KRLE PLZ',
 'Wow kinda funny that thing is very cool',
 'Svesant Bhai if the name is misspelled, brother there is a request to make a review of the Japanese film Weathering with you anime film on October 11 or there is a request to give a little help in its promotion because the basis of this film\npar India The future of my anime movie depends. Please brother'
 'No, that is the civilization of liberals and feminists!',
 'Gundappa malige nikal loffe then, come out in the first time.',
 'This is real love'
 'Do you have the right to say anything?',
 '@crazy Ram aap mesej kijiye watspp pr',
 'Milti ni ladki yat bhaiyo waha kiya baat j',
 'Pratik Bhai, I had come to hear the review of the film..?',
 'Mumbai my number is 8286209041',
 'He is not a temple but a statue is said',
 'Everything happens except Bollywood main study',
 '@Sachin Saini',
 'Knowed before seeing Arjun Reddy that he is innocent..Pure love...',
 'What did Sahil do?',
 'This film is my fav for Shahids acting. and o movie kabis singh like\nnahi banana hai it is made to give this message',
 '@ChristinaMusic you are right',
 'If you were there, what would have happened to us \nThnk you sir ️',
 'Spoiler given. What was the need to know that the pregnant Rati and the child would always be the hero of heroin. Nothing new'
 'Thank you... you give good reviews',
 'Not one. Movie is...best movie of my life...Bas ending chhor ke sab\nKuch is 70% related to my life....I love this movie....I 9 saal piche\nachla gaya was...',
 'Good brother Ranus hope, he said Chhaya.......',
 'I have become a fan of yours or martyr',
 'Submitted Identity',
 '@ak 98..ek there is a channel named "Fmf"..Go get it...and see how anti-Hindu is this Bollywood\n',
 'Daily Profession',
 'Well done everyone should do this only then this law will change',
 'A Dam Right Perfect!!',
 'Jai Shri Ram Vande Mataram',
 'When there is a discussion on such topics or you do not agree with someones point, then feel\soft to use it, but if you are going to have a child, then you should take it all on your own. I mean so much So much for reading 2 million people\naakde wrong h comments of the government\npe this is a topic of more than 10 million h\npe then no one will abuse anyone',
 'Why is it hurting brother',
 'Bhai tune video watching main bhi bol hoon kabir singh is a very baker film',
 'Save the body campaign has to be started now',
 "Sir this is a request please make a video citizenship bill...mum ne smriti h but\nachhe didn't come",
 'Bala pe vi aapka ki hai woh jana chahta hoon bhai',
 'Your saree video must have happened brother',
 'Hindu, sanskrti, ki baat karne scar aj shadi k pahle sex or every 1 minute m\nkiss karne ka support kr h'
  'Sarvesh Yadav is the same brother, that we should work in our own attitude\nt become impatient\nkeep working my brother on your own',
 'Amazing Stuff'
 'Prateek bhai is a big fan of you when it comes to giving honest reviews, but I hate to disagree with you about this film. There is no talk of feminism, the basis on which this film is made is wrong.'
 'You have nothing. Here it is asking for 47 lakhs!',
 'Some people like Om Shanti Om, some like taxi driver, public relations is se\nghnta frk ni pdta',
 'Wonderful brother. Band Bajadi of Liberals and Communists.',
 '@vinay hottie?',
 '@SoumyaRanjanSahuAbhishekBachhan is a very good actor he "Khele\n Hum Ji Jaan Se" Bhoot hi good thi suraj sen, I know surya sens name\n from USI film, this movie was released in 2010 as given in I have made 2015\n .His film Raavan, Run, Bluff Master, Run, Games, Guru, etc are masterpieces. If he\n plays a supporting role in a film, he\n is given good acting in it which goes unnoticed by Logan. People started expecting acting like Amitabh\nBachchan from him, due to which he got a chance to have a different identity\nbnane NHI the way Tiger Shroff got it. \nKar caste is like a stigma.',
 '** The brother did the postmortem of Kabir Singh ** honest review',
 'Natural background light is nice sir',
 'Masha Allah',
 'Where did Abhishek Bachchan come from brother',
 '@roshan bawa ha ji',
 'Perfect h review ️☺️',
 'Kabir is the best film Bhaiya..I see every 1 review of you..bt is time you\n will supote nehi karunga',
 '100% increase in 10 years but 70% of that to abuse',
 'T-shirt is MST',
 'It feels good to be your subscriber...well said Prateek Bhai',
 '@ShivSrivastava please give your number',
 'The stars do not ................',
 'The first thing to do is to put your trust in love at first sight',
 'What do you want to say, Bai'
 'Brother after drinking ganja mount',
 'Rite Bolela',
 'Lets communicate that you supported Arundhatis program! \nSearch news on government \nBhai strong language word address! \nContact Urbanets and Networks in 2020 ? \nRajdeep sir',
 'Saurabh bhai, your friend will not show the paper in Varun Grovers poem nWalle wale pe kya kahenge?',
 'I knew this brother also hates the film has my place, I knew\n if my attitude matches with anyone, then I will fight with the fellow'
 '**Cute Film!** \n **Cute Film!** \n **Cute Film!** \n **Cute Film!** \n **Cute Film!** \n **Cute Film ***',
 'Sir aap jo kahrahe absolutely right ke nothing is there in the film.. I too\n told this to friends after leaving the theatre.. The story is the same Ghasi PT\nlekin is the reason for the success of the film, the way it is presented He\nplayed the character which was very good.. Heroin didnt act properly\nlooked like neutral but shahid did good acting..sometimes\ndoing the same thing in a representative way gives real result or this movie \ nusika namuna hai',
 'Mujhe aisa lagta hai hamara pyara bharat jaise sir bole yah it cant happen\nour it is not a matter of human rights\nthere should be any criteria\n for any job...and our culture is completely different . We cannot adopt some things\nWestern ideology.',
 'Bhadiya Ek Dum Mast',
 'Sourabhi ji and Rajdeep Sardesai ji Arundhati jis full name to speak.'
 'Saurabh Dwivedi ji is not being given anger to Arha bhai',
 'It felt like a lot',
 'This is a remake of a Tollywood film',
 'Elder brother I am working but I am not able to concentrate..\nchata but after a while the mind changes...',
 'Wow brother took revenge',
 'Hey brother, they speak like this \nand have called Lord Ram a terrorist',
 '[14:23](https://www.youtube.com/watch?v=J2J5ssSP5yQ&t=14m23s) Scene ..\nChain break do yaar',
 'And the film worked above 200 crores',
 'Jai Hind ',
 'There is also the possibility of farewell with the intellect in the discussion.',
 'Who listens to those film companions? \nWe go to see films only after seeing Prateek Bhais review.'
 '@MayurDeshmukh I didnt give street to anyone, brother, I kept it straight.',
 'Arnab',
 'There was a lot of need brother this topic.. saved life tamne meri.. Thank you yaar ',
 '10/10 Wow now to see brother',
 'Sabash Lion',
 'Amber de tere ghar par j',
 'Bhai Love M Kabir Singh had also died, had given him career knowledge'    
'Boy girl dr doctor... dude to check with doctor thick... and doctor\nme prb s hai... bat not...'
 ' A person who happens to be like me has done this and hasnt done this'
 'I like a lot of brothers talk, I am also the hearts thing',
 '10 /10 Pahil bar bv'
 'Kabir Singh is a single film',
 'Ye kon discount: written',
 'Oh will you be friends',
 'When will part 2 of Can ** be played** ???',
 'Brother for telling the truth, \ntu really brother...',
 'Jai Ho Brother',
 'There is no problem with true love ... Tell me one thing if you are \ npasand karte ho of a girl, then you are in the first relationship then you will do it recently? Do you love\nthen Marui?',
 'Sir in the identification and identification of the logo \n \n \nEagerly searching only to search for the identity',
 'There is a word for the film - Mahachodu film',
 'whatever',
 'Kanachana 3 ka bacteria do plzzi. \nThe official premiere of Kanchana 3 is on Sun Next.',
 '@Sirfira Kafir 007 Brother fix the program Kyoki Chand to say a hi\nh, Sitara Kyoki Sitara is getting the second one, use the boss',
 'autowale bhaiya < 3 < 3',
 'Your break up cob sir',
 'Useful for modern life... It is very useful for modern life. ...so I can\nv be predicted wrong...',
 'Saree Update'
 'Hope from Kabir Singh Mivis part as he can connect with dreamland?',
 'Very good',
 'Swetaabh was playing like you like.... today you are wrong... meet \nkeep rejoicing',
 '@debashish chakraborty bhai t a number b* **** this is because your name\nnamed Homosep and *******',
 'Brother film is good',
 '**We will see all of your book, wait for all* \n ok man❤️❤️❤️',
 'Madrasakachak!',
 '😂 my friend also gave color to Kabir Singh and finally now brother\nShoom pad started',
 'Fayo Terminology Officers Cooking',
 'Do not want',
 '@INDIAS TOP 5 He is also capable of doing work.',
 'Dude editions of these things',
 '[6:28](https://www.youtube.com/watch?v=N_ZMfQMZos0&t=6m28s) \nOh brother \nmad what \n4 nhi, 2',
 'Sir Apni Daily The Hindu Kariya,Sir Recrest West.',
 'Kabir Singh is good',
 'One step said',
 'In communication with brother in communication with Habib has covered',
 'Sir..Drav Rog',
 'The lights that the society is talking about',
 'Whos News'
 '@execute we are Kabir Singhs "raw plow"....',
 'Where is the movie? \nAnd who is emotional.',
 'The weather will turn hazy',
 'Like haha is big and for',
 'Morning is right the first time informant must change item to item...\nDsara ️️️️️️️️️️️️️️'
 'Then it was made before we entered inside...\n the glory of the country and the outsiders glory\n is made in India!',
 '[3:47](https://www.youtube.com/watch?v=ZzsAuDkXq1M&t=3m47s) I am my brother, I am love..',
 ' The sentence from my bus is... Bipin Rawat ji..'
  'Moody has so many fathers. like',
 'Sam is the Maya of Kaliyuga',
 'Spoiler alert for lentils',
 'Brother article no section 375',
 '@sdr Mishra what are you saying brother...nothing is clear...\nseems to be...Modi government once again...',
 'Then we made falsehoods',
 'I want Karna according to me',
 'And the journey from the maid to my brother changed the language of my brother but the feeling nee',
 'Sucharita ',
 'Show Gay the Bad MT',
 'Please give the review of Dream Girl soon, today I am thinking of going',
 'There is no such side effect that you guys are seeing in the video',
 'Kolkata new movie please\n[#bibaho](http://www.youtube.com/results?search_query=%23bibaho)_obhijan\ntrailer ka response do plz sir',
 'His life had become a baker by implicating jaggery in a false case',
 'Ramadhir',
 'Bhai has taken the entire film night...',
 'Wai One Side Ke Dari Katna Wal Gaye Kya Aur Fashion Hai anyway great\nreview...',
 'Brother, I am that Jats number six years',
 'Kabir Singh ️❤️❤️😎',
 'And jihad does not mean just war or war. Jihad is a thing. If the \ nectar person is struggling and struggling to earn money in every way to keep his family happy in his normal life without anyone \ nnuksaan Pahuchaye then use is also called jihad. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . The media people have seen Jihad in the wrong way. And those V Atanwadi attacks are Muslims\nhi. Because Allah ta\'la says in Quran Sharif that if tm will pay attention to my\nsome innocent "insaan" ....here "insaan" is not Hindu or \nMuslim or Christian, he would have killed a human. If yes, then TM is the danger\nkrte of entire humanity and if you save the life of an innocent person, then you save the life of entire humanity.',
 '@prashant kumar somesh mast h [dude.bt](http://dude.bt/) this bad content\nh.bhot achi Movie h',
 'Of course India must be me',
 'Horror tube and how do you understand at first glance, brother? Just a compromise...',
 'What was the need to dance item song in Saala Super 30 in that also item song',
 'The brothers who disliked them..say something in their naked...',
 'Sale sale',
 'Eat right brother... will enjoy watching the video...',
 'Congressmen and Aam Aadmi Partys are spreading riots, also tell me this. \n \n<https://youtu.be/8BGuVdwj12U>',
 'College mein sone launda hai kabir singh wali dp',
 'Was waiting for this'
 'Okay brother really',
 'Half more rape cases would have been false',
 '@Bharat Behra Nahi Do',
 'Sach kahun to daily current affairs daily aati kab hai?',
 'Kya brother so night mai .... what baati',
 'I will say ** tik** ** tok** ** become ** do....everything will be fat...😂😂😂😂😂😂😂😂😂',
 'Shahid Khan nKabir Khan...',
 'Speak right brother, India will travel towards negativity with such a law .. and\nFirst of all there will not be a law to protect from negativity..',
 'Ooo woman \nfriends cringe?',
 'Jagkanambatau',
 'Adi came after shaving',
 'Brother, drowning in your sleep will be a habit',
 'Jai Bhim Sir\nSir please make a video of Daily Prelims Booster...',
 'Who are these liberals, from where have these people eaten',
 'MTLB can not join the army of tik tok',
 'Another dialogue\n\n"If Tiktok can be a hit in India, why not Kabir Singh?"😄',
 "90s movies better the, waha bhi item songs but one control\ntha. Nothing was visible. India is a strange country where people\nlive their lives watching movies.",
 'Sir pick the next boy..we support',
 'Swetabh brother, you who have come to the society, you are doing the right thing... boys\ nnijaljao is from water ....',
 'Tomorrow 100% Dekhna Ka Hai Bawa',
 'How will it be possible for me to set up a pen drive course for railways Assam',
 'Good son Arundhati pe ye',
 'Reviewing Brother Dark Season 2',
 'Brother, you were looking like Kabirs brother Karan',
 'Brilliant brother, give him a big slap.. I agree with you',
 'Sir Please Top 10 Best and Worst Movies of 2016 Pee Ek Video Banaya Na \nKafi sam sa bol raha ho plz bana do na \nplzzzzzz',
 'Bhai full movie bata di chutiya he kya',
 'I never thought of watching it. \nKunki I was told the story by the trailer itself, which I did not like at all.',
 'America itself is alive on the talent of the country',
 'Only Arnab can teach such people',
 'Shahid Khan',
 'What is a masala masala for the general public to burn a scooter and a car',
 'Morning Thing',
 'I am also completely devoted to my love or even my partner\n... both of us are bhat happy',
 'Spoiler Free Review Bana',
 'Kya baat hai brother video pe video',
 'O allah mughe shallow',
 '@ Pacho Roy Chowdhury but in that \nUnhide your subject full address \n..or gf b thi to ayyaan(IPS Officer) Ko bta ri thi kya krna h. \nVery she movie h...',
 'The big thing, brother, is to define it...',
 'Bhai aapko TVFs Kota Factory web series dekhni chhai bro',
 '+nishi 76 \ngoogle kr lo yr',
 '@L Lawliet lagta h bechara bhakt h aise tell ja na',
 'sale billary',
 'and you?',
 'Jio Y',
 '@PraveenNimje Gujarat',
 'One thing is the year. \n[2:30](https://www.youtube.com/watch?v=N_ZMfQMZos0&t=2m30s) min. ki movie me\ndirector\n\nkaisi,\nreality will dikha dunga. \nhe will get only Krishna,\nye one look love',
 'Very hard bombshell you are right\nheartfelt respect for you🙏👍😎',
 'What work was to be done?',
 'Bhai was a very honest and logical review. greet you',
 'Brother what are you \nwhat are you doing\nwhat are you saying',
 'You two are saying wrong Arundhati Roys bare me.',
 'Defense me homo ho nahi hone chai bkoj if any army jawans cant \ nye pta chala k unlogo me se koi homosexual h to shayad baki log us se\nd create a difference or they (homo) thick se army me sarva kare ' ,
 'Brother has made a game box',
 'Sahi Hai Bhai Best Hai Kabir Singh',
 'Is this movie review the??',
 'Whenever the crisis came, it also came like you. \nYou have a true saying.',
 'Brother, you didnt give the A-rating right',
 'Kabir Shing is the best movie ever in a love story film, which is a heart-touching film. Kabir Shing will be a superhit blockbuster film so far',
 'Hero Ji',
 'Hon Chie',
 '@PranavSingh :)'     
 'Sirf Uri mein item song was not Syed',
 'Before Arjun Reddy came, he didnt see it BC',
 'Pratik Bhaiya, do yoga, the weight has increased',
 'True Symbol Va.👍👍',
 'Hey what is the matter with NRC',
 'Sir you are very nice, wish I could meet you sometime',
 'Membership of the Hindu family. Wifes companion can live\n sap life',
 'Semma Dialogue Dhidhi',
 'Yes Movie Masterpiece is Indias Best Court Room Drama Movie Yah Comment\n If you have any post please support the movie',
 'I become your fan, I want you to have 1 million subscribers\n very soon',
 'Jio Dada',
 'Dek Bhai Movie is awesome..\nBut those who follow this character will go...Woh To Gaya..\nBut dont spoil the movie. \nthis movie is the biggest all time blockbuster of 2k19',
 'My heart ki baat boldi bhai ne...',
 'Thats another tandoori.',
 'Di Ke Hi',
 'Fault Ik level of that, which Bollywood and Salman gave, they gave India some good\nmovie society not too many people',
 'Very well explained sir...',
 'Subtitled Nahin Hai Kirik Party Ka... Voot PR',
 'Great brother'
 'Eat right brother',
 'Spoiler alert for those who havent seen it, dont watch this video',
 'And sir why didnt you review the Qarib Qarib Single Movie...',
 'The groom has also planted, they can potentially put in red. These to\nRaanjhanaa film cameras have done the same, as they have done so. And the potential for spoilage,\nWasnt it artificial, The capacity for presentation is also high.',
 'fun',
 'Sexual emancipation is a favorite subject of liberals and they just want to humiliate men\n........... the perfect symbol Siri',
 'Movie Super Hit Hai Guru',
 'Bhai Prateek, Liberendu and Feminazi To Gaye Bhad Mein Magar Hai There is no such thing as love in the entire film\n I completely disagree with you... Is\nfilm mein ka koi sir hai na jodi hai ... It is not right to say that any film is good just to oppose Liberanduas... It seems that the \nugly child has been found between\nDevdas and Rehna Hai Tere Dil Mein with an illegitimate relationship in Kabir Singh... I like more than kisse,\nFour more shots not even four people would have seen it...',
 'I adore you a lot..I listen..Keep the political side from review..pls',
 'His marriage will happen, brother is single.',
 'Saying completely',
 'We watch ISKA channel for BS entertainment',
 'Love you brother!! All work in progress!',
 'Bhai Uri The Surgical Strike Main Nahi Tha Item Song',
 'Phaddala brother, what a wonderful video nLove you 3000',
 'Army Waloko Nigam Physical Divine Sankaris some convenient \nKarvini if ​​it should be so.',
 'Give your address then. I send myself.',
 'Like without watching full video',
 'Very well.'
 'Very good friend. The same thing happened to me but I could not change like you\n, because I would see the old age of my mother and father. If this is wrong then anything I can do with this topic. Today I am happy that my mother and father are \n. With the wrong app.',
 'O woman, will you make friends?',
 'Right to live your own life',
 'Should have given an A rating',
 '[19:17](https://www.youtube.com/watch?v=N_ZMfQMZos0&t=19m17s) was ahead',
 'Half the universe of Thanos is the biggest reason Karna says you\nre a "feminist" and a "liberal'
 ' Laski Choudhary also gave a must reply inco! And you too! A hero\nbe...',
 'right brother' 'the friend knows that there is no point in the chutiya',
 'What to do, does not fill his stomach. Lets eat.',
 'This is the best of the year. Saw it yesterday'
 '@madhusmita pradhan yaar dont take galia seriously... thats just an expression of bhadas\n exit...',
  'I will do'
 'Shahid Khan \nno Kabir Khan nor Shahid Singh \n😂',
 'Salaam to you my brother, I wish I had the courage like you',
 'Solute Brother',
 'T-shirt only took feminist kiss',
 'Keep your mind open brother',
 'Pakau Review...',
 'Part 2 Chaiye',
 'Is there tolerance in the ears of the soil of the country... We were attacked by thousands of foreign invaders, millions of Hindus were massacred, forced\n conversions took place, Guru Tegh Bahadur ji, Sambhaji Maharaj ji, Banda Singh\n Char Sahibzaade such Countless great men have given their sacrifices\nto protect the culture of the country...\nYet the country never discriminated against anyone..in the whole world\nwhen the Parsi and Jewish community were persecuted then they were the only and only\ nWere safe in India... Is there tolerance in the ears of the soil of the country! \nAnd this country is a Hindu majority, so it is secular! \nYou please do not judge our India on the basis of the history of other countries\nkariye 🙏 \nAnd you should first clear your left-right concept... \nWatch this video - <https://youtu.be/yNdB772uASU >',
 'Or Surv, you have changed the statement very clearly. Even taking that dirty name\n I know ghee, chose it and said Home Minister KO Ranga Billa.\nYou correct it please.',
 'What has happened to the world?',
 'Sir being homo or n being someones bsm is not a \nye to god gift h so i dont think it has to be something that\neveryone should be mtlb by their work whether they are fit or not\nso army m should also crna must allow',
 'Hello Patrick sir\nRambo movie all part in Hindi whats that? \n& All parts are available to me on any one platform \nPlz rply',
 'Say right bat brother',
 ' @ Suraj Kumar you are very literate man. What a good comment you write.\nYou look like a topper.',
 'We accept LGBT and gay. I thought he had a human',
 'Little 95% of Indian Army is already gay',
 '1 day I uploaded 3-4 videos with our elder brother n ak today so we\nLundos happiness has no one',
 'Bhai dil chuu liyaa musttt j bilkul',
 'Quite right'
 '@AbhinavRajput not like that but',
 'Hey or brother 375 Mavis is not like this on mobile',
 'Hey Upasana Vishal', 'Opening',
 'Sir Koi Mil Gaya and Gangs of Wasseypur also inspired people',
 'I thought I liked your comment',
 'How many ads changed man',
 'I dont love Jim'
 'Right bro, in the coming time boys will need reservation',
 'Why was Sujana Arundhuti Roy not put in jail till now??',
 'Well done.... otherwise you guys are having sex in war time\npe...... I only have sex, sex sex...',
 'Yesterday only saw half the movie and that girls point is perfect... half because \nwatched the movie because its a chutiya',
 '@aggressive squad ha brother, but when this review was not hi, then the video k\nbahar movie review q written only discussion would have been written',
 'Very nice',
 'What happened Aman Dhillon brother, is everything fine now? Please contact',
 'Pratik sir please tell me some Salman Khan movies in Bari like Tube\nLight, Sultan, Bajrangi Bhaijaan, Ek Tha Tiger, Kick and Dabangg,'
 'For the first time brother did show off in the first line',
 '@AmritpalSingh ok oh where do you live',
 'I unsubscribed some time ago.. but now I realize that \nopinion only strengthens the country \ndifference to a subscriber.',
 'Light went off in our place too, but it is not running less, he keeps on passing the time every day. Daily light caste that. From Kutch, Gujarat.',
 'Perfect brother... set it on fire...'
 'Pratik Bhai... in a pure 16 minute video, didnt tell anything about his movie... what about screenplay, editing, camera work, acting, music\net... then what a review it is,have made its own headline...Kabir Singh\nReview...Lets tell a little about the film like a film critic...Baki\nSo everything is fine...😊',
 'The movie average j... is not even that big.',
 'Pratik Bhai, you are right to say but the review and discussion becomes very long\nYour, as well as the technicalities of the film like music, dialogues,\nscreenplay, acting, direction Jan Toh Baat Hi Nahi Hui',
 'I didnt even review the movie, brother'
 'Sir remove CBI, on Alok Verma ji...',
 '98.2% of fake dowry cases in India 73.3% of false rape cases as per NCRB',
 'Whether gay or lesbian doesnt matter, just focus on work, any country m problems no h to yaha q',
 '@AdarshSingh\nBhai Teri photo I felt in first turn that Hindustani Bhaus photo h lol',
 'I have seen the movie, it is a very good film',
 'Okay sir!!!! Sir have not uploaded the current video of 9th and 10th January 2019'
  'I kept a watchman at my house, now that watchman says you prove that you are the owner of the house.',
 '[14:39](https://www.youtube.com/watch?v=N_ZMfQMZos0&t=14m39s) Mai says this is\nchutiyapanti.',
 'Hey... what do you want to say? ',
 'Should have spoken spoiler the brother',
 '@Vinay hotty good thing h shaadi kroge kya mujse.',
 'Very cool bhai superbaby',
 'Will kb review brother KGF?? \nI watched the movie only after your teaser review!! ',
 'You only have to review the same film or web series in Hollywood dubbed in Hindi because 80% of the people in India watch Hindi dubbed film Hi',
 '@praveen nimje number de',
 'Three meters above the sky. Review Spanish film..Bollywood remake will be\nso good.',
 'Urbanbhai why are you making so much noise? We only decorate the drama. If you\nha the channel is chaotic, then the family has said, why are you losing it?',
 'Your eats well in Hindi and in English.',
 'End wala banda was cool... let him also become an actor of timeline..',
 'Review of Sir Plz Hackshow Ridge Crow Na Plzz. Yeh bhut hi shandahar movie h ek\n bar dheko sir please',
 '**[14:00](https://www.youtube.com/watch?v=N_ZMfQMZos0&t=14m00s)** **One number\nline is spoken brother....**',
 'Thank you so\nuch for watching your video my face pr always smile hot',
 'Oi, what are you doing? \n has changed \nOoo whats up my brother?',
 'Sai hai brother',
 'Whatever J Movie Copy Paste Hi J',
 '@KRATZZ Sacred Games Season 2 Bhat Achcha J... BSS Those who watched the web series NI...\nthey remembered',
 'Do you need to give everything m feminism?',
 'Sir you are hard wale youtuber nikle 🙏',
 'Okay but keep it private people tell it publicly',
 'washed',
 'If someone wants to serve the country, then what is wrong in it... but we have to keep our countrys honor high above us and it is the duty of every soldier that his inner soul should be pure.',
 'Oye party ji... dosti karoge... from mugha...sunil dewan..98206 63694.',
 'Absolutely heartfelt',
 'Great Review Prateek Bhai.....Blockbuster Kabir Singh',
 'Kabka was waiting for this...\nWhy didnt the Kabir Singh PR review of brother come yet \n....Love you brother...',
 'Mast film we saw...chupchap screamo this film',
 'Bhoot Shai Prateek Bhai',
 'Mansutra Bhai Shab Masturbation Kum Ko Kabir Khan',
 'Finally...somebody thought like me..\nThank you Shwetabha Bhai',
 'Very satiric review, even if I watch the film, I definitely watch it when your review comes. Why the way you explain the philosophy of the film, maybe even the director of the film will not be able to do it.'
 '**How ​​many found Kabir Singh movie unreal?**',
 'Brother, dont make such long vedios, dont repeat the words, you are boring',
 'Arundhati Rai is saying that ?',
 'Ranga Billa Modi \nwhat is the name given.👍',
 'Whole channel only one video very disappointed and put more videos put more than this ka\njyeda',
 'Review Sir Rocket Man is not the biography of Jhon Elton, he himself gave the music of Lone\nKing...plz 🙏🙏🙏',
 'What untoward examples diyas, you should have a plan... uncles\n... like in your channel also understand in easy language...',
 'Bollywood also sometimes makes good films',
 'Bhoot was happy to see this news',
 'When will you upload the Toy Story 4 review',
 'More than 1 woman is arrested on the report of one woman.',
 'Ha brother fuck boy only',
 'Respected Desai ji the solution to these problems',
 'Napurrri',
 'Absolutely Correct',
 'Gundfad!!!'
  'Jazeeb Ali',
 'One thing true brother',
 'Not a movie',
 'Joker 2019 Bhai',
 'Brother, you are absolutely right.. I agree.. If there is one thing about equality then \nthink both ways',
 'I had no expectations from the film, but after seeing your review, I will definitely see it',
 'Review the movie Bhai Chhichhure too',
 'Who Decided?',
 'R Year Is Bar Mein Also Chutiya Ban Gaya, Thank You Brother apne Eyes Opened On The Picche Of The Movie',
 'Wont it be a while? Thieves what \u200d♂️',
 'The right symbol, brother, sometimes people have a lot of problems with a good film',
 '@HemKanwar are you a Rajput??',
 'Ranga Badla',
 'It doesnt hurt'
 '@Aurora I also see the review but....aap sirf film ke bare mein baatne\nna....',
 'The game is Chandu Bhais great acting. Chandu Bhai did it......\nrho...will go ahead......'
 'Give Bernol to the Liberandus',
 '@crazy Ram 7061030501',
 'Seen in Kabir Singh that sex is not love when he has sex with another heroine\nwhen he says I love you, then he shortens it to him\nit shows how much he has committed to Preeti and when If two people love \nsomeone, then we can not understand in their understanding, my friend and how much he\n girlfriend brings, but later gets punched up',
 'These come under the category of san julus like Kabir Singh who are \nfantasy story tellers.',
 'Wah bhai I requested you for Kabir Singh movie review',
 'Realistic film doesnt work friend',
 'Pratik Bhai your review is great',
 'If Aarup is proven wrong, the girl must also be true',
 'Is the plast'
 'Arjun Reddy is also very good to see',
 'Absolutely right sir'
 '@JigarKumar will do NBR',
 'How biased is this video...',
 'After what he did, his life was also ruined, neither he is in jail nor will he work for 15 lakhs in 3 years',
 'These people should not be in the army',
 'correct..',
  'Now you will cook',
 '@ش سين said right also your',
 'What did you say man? I washed everything. \n\nKeep it up brother.😣🙏👍✌👌👏',
 '311 Dislikes. Feminists and liberals did it',
 'You are already fat',
 "Sir please review Malayalam movie Premam\n it reminds me of films like Geetha Govindam and 96",
 'Look at the label of the disorder',
 '@keshav is right',
 'Awesome symbol brother',
 'You are doing lotus brother',
 'right thing h',
 'Did the brother half shave?',
 'Sucharita Flim Saathi Ke Naked M H',
 '@Jayesh Patil'    
 'What he told in these words. The message was wrong..\nPlease do something new, we have no right to ruin someones life...',
 'All this is an Urban Naxal.',
 'Bilkul Sahi Bol Re Ho Sir Saath Saath',
 'Bhai e gaya tu take Army of Titans',
 'Mostly wrong use',
 'Maybe after divorce',
 'Etna yo was shocked after seeing the trailer....G',
 'It is right not to be at all, guns humsa shoot on the wrong wake',
 'Bhai Kabir Singh Dekho Fantastic Movie Dil Khush Ho Gaya',
 'Jali a yaar',
 'Brother you are from Mandi',
 'Sir mi arjun reddy movie hindi mi dekhna hoon kha milegi... please sir',
 'I was very much waiting for the review of this film, thank you brother.. wake up V I will be happy to see this video of yours...',
 'Shwetabha Bhai ️',
 'I dont know if the thinking of the Indians will increase, or if the General of the JB Army is saying this, nor what can be expected from the people',
 '@Truthshallsetyoufree mujhe Job dila do IAS office.. fir main christian ban\njaunga bol',
 '@max Monika again the same mistake... you are doing the \ntulna of Indian Sanatan Vedic culture on the basis of the history of other countries...\nAnd yes... the facts I have told are 100% true.. . You read a little history... \nAbrahams religion, Sanatan Vedic culture, read its history yourself...',
 'Anand Agya Mausam Bangya',
 'Sandeep Reddy Vanga Pakka would give Shwetab a gully',
 'Bhoot Badiya Acting Chandu Bhai',
 'Hahaha thats right, if someone per head, only pulse is power.. Thats enough.',
 'Shi said brother',
 'Brother, today Ankur Pathak looks exactly like',
 'It was a chutiya film',
 'Its right sir'
 'Love you bro... made such a good film a flop show \nwe were watching so happy, its such a good film...but from here\nthe perspective has changed'
 'You think too much'
 "When the brother said 'the heart felt happy seeing the liberals burning', liked Tabi's video",
 'Bhai no one says anything but Kabir Singh is a masterpiece film\n just saw the whole paisa vasool...',
 'Yeh Hota Hai Real Review... Bhai when is your book coming',
 'Everyone is running their own shop, brother. But you took his',
 'First hit us with a lot of lilies',
 'Yes brother, I want to see for myself how my friends follow blindly',
 'Buckwow movie kabir singh',
 'If I didnt get a girl in the end, it would have been realistic',
 'Reviewing now'
 'Its worse than me',
 '@ AK BAHI Ayushmann Khurrana Saree Movies My Romance H BC BSSSSS',
 'Bhai this movie is a super duper hit in South and also in Bollywood\nhui.......it shows how many % people need you in India',
 'Bollywood was always chutiya',
 'There is no use even by keeping the pub people that way',
 'Brother brother brother brother',
 'Sach Main Jhand Film Hai',
 'Kabir didnt allow Preeti to do anything, lol or dont do anything about it\nrhaor now brother said what I was thinking good review',
 'Brother, in Bollywood, people would hit the same film in which you hit the emotions and\nt also the actors are overacting',
 'Brother, you are the only one who has the power to speak the truth',
 'O woman, will you make friends???? Bhai Tere T-Shirt Pe',
 'Pratik sir review Money Heist Series',
 'Completely agree. Jai Shri Ram. Jai Maharashtra',
 'Those who are calling the comments me homosexuals are bad, they should not\nkrna because it could be anyone who is calling them bad and bad, maybe their children should also be paid homosexuals. Did you feel bad. But maybe\nit may happen that this creation is not created by us, it is Gods creation, TB what\nwhat would you say. Whether a choice is paid or right or gay, it is not their fault.\nPr unse nich. As soon as your mistake is wrong,\nhme should give them sympathy for hurting them. I still know that you will answer only with the words \nPublic relations still change your thinking'
  'Tu Ranga or Rajdeep Billa',
 'You look more cute than a hero so sweet',
 'I have a question. Ager futuristic ho hog to Larki lok kisse love slack? \ Ngày Hona Buri bat nahi hai par yesa ho ko namaste chalta hara ase. Those who were gay\nhai they are not gay for love, they become gay for sex. \nhai whats this good? If there is really love, then show love by stealing sex, then Manu Tumlok\nReally different from normal people.',
 'Sanatan Hindu Dharma Ji Bhai...Very good...you guys will wake up Hindus together',
 'Brothers one no. Picture J must go to see the year',
 'Where is Mobarak sir',
 'Sir, my heart got a lot of happiness from your words',
 'India is the worlds biggest democracy, where we people talk about equality,\nhh then we prove this bat.B will be a masterpiece by treating sb as equal,\nin the name of gender\nKeep someone new from serving the country And apart from this the gender of the people\nzyada should focus on their work. Thank you so much sir'
 'hii.kaise.ho',
 'Ha yaar was a very cute film. Everyone was saying that Badiya was a film, so\nsaw bt bilki jhand tha',
 'Do those who travel with Jaguar need to get a doctor?? ',
 'Everyones acting is good',
 '@Anil Bhargava are bhsi vo to thik hai lekin in separate video of things\nlo',
 'Bhai Gullak TV Show Review Karo',
 'Sir we are human beings\nlater men, women\nand both can commit crimes',
 'Review Bhai Sahu Move Taser',
 '13 Reasons to Review',
 'okay brother',
 'Brother, do you know Lakshya Chaudhary? \nHe is as good a tuber as you',
 'Brother, why do you talk about Eder there, man is boredom brother',
 '@RahatAhmed Hey so take 1 movie like a movie, its a bit real life\nhe, should it be Laila Magnu in all films??\nhes bad in doing something different????',
 'He was expected and said Hahaha:',
 'One more thing- In this Kabir Singh movie, Kabir Singh put his family at stake in the affair of a girl, spoiled his brothers marriage...',
 'Brother, you speak a lot by making a little video',
 '@RahulSingh Bahadurgarh, Haryana',
 'Punish the rapist. Punishment'
 'Even if not realistic, then to make Golmaal type mindless, neither did I enjoy it',
 '@AnmolGoyal koi toh firki lera bhai',
 'I also like Rajiv Dixits video',
 'Okay you have said, thank you very much',
 'Brother, I do not see the cup of Namo again?',
 'Thats why they say dont comment without watching the video...',
 'Like it is my birthday',
 'Sarvat was lacking...',
 'Just abi 2 hours flame mako pata chala, one of my friends left the girl or now\nhe was PD in daru sute or he had felt the hand vaccine of fruit kabi daru'
 '@vamshi bommena',
 'It is as easy to live as to call someone a Nazi',
 'Brother, you did the postmortem of the film. But what he said is absolutely\ntrue. \nBrother, tell me the name of some good movies not worth watching.',
 'The last line was right brother "If I understand, some of these things can be gone \n:p"',
 'What is the matter \npure year sona ni hassa mai \nso true',
 'love like a dog',
 'Thats why it is said that without watching the video, do not comment! \nWatch the video first, you will get all the answers! \nKabir Singh and Four More Shots Please they cant be compared!',
 'Bhai uksa background music and ka animal bhar ajata hai..',
 '@ArpitJaiswal',
 'Narendras effectiveness is rather gruesome to the Left, which spoils Indianness in every sphere of our society.',
 "Sir 'Chhichhore' from the movie Nade Apna Rai Digye...",
 'Ha Me Hoo',
 'Half the point is not understood.',
 'Biya become a leader... you talk more political than films, brother',
 'me too'
  'These people are campaigning for idol worship',
 'Vada va v vada v v vada vau \n \n \n \n \nI am good in the film',
 'Yaar movie jeh koi reality nahi hai',
 'Man friendship croggy?? Lol',
 'Bhai TM larki hote sam se baba PTA was doing... Love you brother',
 'Super Sach Bat',
 'Abe so kun on a 3gante ke film. bolta ha ho movie ha deku enjoy karu\nkattam karu',
 'What would have happened to auntie...',
 'Kabir Singh',
 'Done right, done right'
 'You are at the forefront in support of South Movie, Robot and Bahubali Movie h\nfuddu Banate h, Always One Man Army Heroes',
 'Haji',
 '@ShivaniMahajan hai yaar all this is lotus of your fake DP',
 'correct',
 'Yes Shwetank Bhai, I need a little improvement in your anchoring\notherwise you can be a good anchor and become very good',
 'Nonsense......this video is very helpful\nhOAG in keeping society against LGBT',
 'Brother T-shirt is a must',
 'There was a review...!!!!!!',
 'One like is made for brother',
 'What is the matter Prateek brother so scared night upload अपलोड you are going right brother ji was just\nsoch seeing what you have come brother © Support \n **Vaibhav Vlogs (YouTube Channel)**',
 'Joker Movie Pe Review Bana De Bhai',
 'Sir interstellar review plz \n..\n.mujhe bo movie dont understand ️♥️♥️Love you sir...God bless you',
 'Yes, all homosexuals should have the right to do the military',
 'I have just come to watch the film and went to Beth to see your opinion\nThe movie is very good and your points are the same but it is also okay that our culture is the only believers on this. I will not like this movie\nI liked the story very much but it would have been better if it was\nfinished on sacrifice. bcz the boy who married Karti his life\neven he had to be barbad, it should have been shown that Endonokes relationship should not be the fault of that\nthe society should not do it to the girls as well as to the boys\nIs someone else in your love Not getting destroyed \nAt this point even a film is very important that you leave everything in your love\nsee what happens to the rest of the people',
 'What about these Ganas? \n1\\. Veere Di Wedding. \n[2.one](http://2.one/) and shots \n3.Lust Stories. \n4\\. Dirty thing.',
 'Its right boss'
 'Nice is in nature the opposite of all...'
  'NH Bunty is bad even to harm himself, the man is weak. hai\nya fatten hai fith hai shakalbhi hai haijati',
 'Which is right, brother, people like only the film against the countrys opposition, the one who became Salman\nKhan',
 'Jeetu Vishwakarma Hindu',
 'Maa Kasam Sahi Mein Bachod was a film BC... even today proved brother\naapne😂😂👌',
 'Ranga Bila of Lallantop...Saurabh Khajdeep.,...',
 'This is a remake of Telugu film not Bollywood\n[bhaisaab.Movie](http://bhaisaab.movie/) ka director is also the same\n[tha.Movie](http://tha.movie/) Baker again Too',
 'If someone has to be arrested, then 498 A Mai then only the husband should have hair because only the wifes reaction is the husbands',
 'O ironing friendship',
 'Bay Yaar Aur Wrong Side Raju Dekhee',
 'autowala dil le gaya yaar',
 '@AvinashShewalkar he wants Sanskrit',
 'Acha mai kuch bi bolu kahega tujhe bhai... ki PM bol raha hai... Sanjo\npahal bat ko...This is being provoked...\nbi to those who peacefully protest. ... leave those who protest violently',
 'Absolutely right brother... I also have a bad mind',
 'Chudain',
 'Brother you are doing a great job, stay tuned Prateek Bhai',
 '[4:36](https://www.youtube.com/watch?v=fQbbWQ9ol_w&t=4m36s) if the acre of rape\ndusre desh se km h tu iska mtlb ye ni ki wo glt ni h...' ,
 'Sukar ho someone showed the reality of this film',
 'Say something to me naked of the devotees',
 'Brother T-shirt',
 'Ha ha ha ha',
 'Brother, after watching your video, now everything is set in my life, I focus\non my sphere...'
  '@shayari and whatsapp status ha ji',
 'Easy brother.. make a video worth sharing yaar...at least review the video',
 'Ya, you have chosen the dress very well from now on in the movie... ',
 'So long brother',
 'When will the brother review Interstellar...',
 'King Ch',
 'Very good. good',
 'Bhai movie is very good',
 'Madam is also having trouble with screen time.',
 'Good sir keep it up',
 'Hes not going to help every day'
 'It was nice today,\nThansk started saving the streets',
 'Contact for Mumbai Chakala Body Massage',
 'Shahid Khan?..🤔🤨 \n. \n. \n. \n. \n. \n. \n. \nFamily Man Dekhni Na😄😁',
 'Watch Tumbbad Movie \npe Whats Brother?',
 'See the news of tanker explosion in Unnao',
 'Jio brother... I spoke my heart !!!!!',
 'Brother you are great',
 'Great review'
 'Which rocket science has been applied, brother? Compromise Zara?',
 'Mast acting hai 3 no ki... \nChhanadu bhai super \nPalkesh is also looking at number 1',
 'Sir jee t shirt from where is it late',
 'Wow brother, its fun, who said the real thing, won the heart for the year',
 'Kabir Singh was a very good film because we have learned a lot from this that\n strings people are ruined love m ise child ke rehna zindagi m',
 '[11:14](https://www.youtube.com/watch?v=J2J5ssSP5yQ&t=11m14s)\n[12:00](https://www.youtube.com/watch?v=J2J5ssSP5yQ&t= 12m00s) Excellent score\nby Prateek Bhai. Its just fun.'
 'Yaar Climax Me Too Ashu Automatic Nikal Taaj',
 'Has brother quick review already uploaded',
 'Extra Ordinary Journey f Fakir Review Brother... Dhanush is favorite brother',
 'Thanks sir... I book tickets now',
 'Bhito',
 '@VijaySharma is right',
 'Live my lion.........',
 'Kabir Singh is a very beautiful film... 2 story to the people who love\ncommitted to Imandari! \nNonsense, One Night Stand, multiple partners are being promoted today\nIt is very rare to see a beautiful film like Kabir Singh...\nAnd most importantly, it is not glorified to drink in a film Gaya\nLike Four More Shots Please... Drinking in Kabir Singh is seen as a bad phase of Kabirs life!',
 'Freedom to speak', 'right to lie',
 'Climax could not see what was brother',
 'Pratik Bhaiyya........ Sucharita Tyagi of the film company had called this film badly\n.......The befitting reply you gave...it was a \ntamacha on his cheeks. .....hats of bhaiya..... well done you answer these hypocrites\ndekar👍👍👍👍👍👍👌👌👌👌👌👌',
 'What is that someone from Bangalore',
 '@pankaj solanki awesome bhai!',
 'Mast j sir \n[#Drxpharmaclasses](http://www.youtube.com/results?search_query=%23Drxpharmaclasses)',
 'Aarey guys, why have I been on the sidelines of these people, do your work ahead\nbaro'
 'Which area do you live in',
 'Brother, be a group, we also want to answer the Left',
 'You guys join hands with Rahul Kedia bhai...',
 'Didnt hear Rajdeep said that he was undemocratic Tariqa',
 'This man would have made a different brother \nsama if you are brother.',
 'Shahid Kapoor should watch this video..Phirse morphine le bhand ho\njaega😆🤣😆😂keh k le li bhai.👍',
 'You are absolutely right, I liked this review too \n<https://www.youtube.com/watch?v=3-k7BpiO0kY>',
 '[16:50](https://www.youtube.com/watch?v=N_ZMfQMZos0&t=16m50s) \nOh tabi toh main kahu tum kahan di ',
 'Pratik ji requests you to explain the important part of the film from km to km, dont do it Nisse the enthusiasm of watching the film decreases.......😒😒😒😒😒',
 'In films like PK, only making fun of Hindus, talking about any other religion\nnahi. \nBy the way, Prateek ji reviewed NE Toy Story 4, there is no politics in it.'
 'No mean no.. right brother.. real passion follower',
 'Very good brother very good',
 'The Ranga Bila criminal had earned fame. It was a bad day',
 '@Hydrich Himmler is right',
 'Ajay Thakur. You are from Mandi',
 'Bhai please Passengers 2016 movie review please',
 'Mujhse bhi bakuf laggi diolaug nahi hai girls in full film',
 'crazy',
 'I got bullshit brother......',
 'The name Papu was given by Kumar Vishwas not by BJP',
 'BC never seen such a strange explanation man!! Btw me bollywood\nmovie.nahi dekhta kabhi and ye kabir vibhir chutyapa bhi nay dekha bc but now\nmann hi raha hai bhai ka expulation Sunke !!bahot hasunga movie dekhte\nsamay😂😂😂😂😂😊',
 'Everyone has their own sexual orientation, there is nothing wrong in that...',
 'If you want to see a toxic relationship, then look at Shahrukhs result, it is not a love story\nbut it is an acting class',
 'mast',
 '@NeilPagare bhai because nothing says logic here',
 'Aranu Di is not your answer',
 'Brother clown look man you',
 'It is necessary to show Pratik Bhaiya film Toh Bhoot Achcha J, Love Story Bhoot Achhi J, PR drinking or so much more, even without it it is necessary to show Dikha Skate the.',
 'Perfect Kera didnt do anything wrong. When people like Najina Da, they are going to do this',
 'Everything is right but Pratik Bhai, what is written in Kabir Singh Bade\nhe is meditating',
 'The magazine Bhai M Akshaye Khanna jis fan hmm dil se wanted his films to be good\nchale',
 'Yeh jo ko ye Congress doing BJP oye ye jo kore din jo ko rahe hai \n<https://www.youtube.com/watch?v=4QbK77bSa4k>',
 'Coat collection is quite good..',
 ' Oh good game... \nI mean in the movie, any case should be seen on exterfa,',
 '@AvinashKumar is right',
 'Girl should be punished for misusing 498A I am 90% disabled',
 'Must have liked the app.',
 'This is very true brother',
 'Long live Rama. Bagot Badiya Prateek ji just put less advertisement..',
 'live brother'
 'Too much overacting'
 'If homosexuals have got citizens in our country, then everyone should have the right to remember\n on the basis that homosexuals should also get a chance to play their role in the defense of the country',
 'Brother hear \nSalute to you ',
 'Wah brother, I am not really teaching the world by joining you',
 'I feel that in India everyone should have the right. And everyone should get the opportunity to serve the country. What do you know? In a war like Mahabharata,\nAny crest is found and he wins the war?',
 'live brother'
 '12.25 spoiler given',
 'You are absolutely right. ..',
 "Girl leaves for his dream locked in Super 30, it's true\nMotivation...!",
 'Well said Bipin Rawat sahib...',
 'You are right',
 'I didnt talk about ones brother, Im your big fun brother',
 'Ha brother I am from above',
 'Everyone is right, those people live their lives, we are ours',
 'Movie review toh ki hi ni... what you said is fine bt h..but you should make 2 videos\n...make a pure review',
 'Critics Mafia is the wing of Urban Naxals',
 'Bhai Pura dont boycott good films too',
 'People learn from movies, both girl and girl society can understand the same universal truth and this thing like me 21s laudna',
 'Brother UP 11 is my number heres of Saharanpur'
 'Shwetabha Bhai, your debt will remain on the generation...',
 'Which happened in Amity',
 'Modia',
 'Soorma!',
 'Kakak Bhai',
 'What are you doing brother',
 '@K Saha',
 'Long live Rama',
 'Only Bharate adopt Sanskrit which teaches the poor to live',
 'Get out there'
 'Perfect brother'
 'Film news is good to hear from Upasana Mama',
 'Isme Movie Review Hai Kya',
 'College ka lrofesslr deewana hai he writes erotic thriller',
 'Bhai Sari Life Single Re Lunga Per God Never Come Feminist\nLife With FB or YouTube',
 'Kai bar is more fun than watching the film after listening to your review',
 'Amber de hahahaha right said',
 'Film partner gave the right answer...',
 'App Filmi Campaign Ki Baat Hai',
 'Well Liberal has got a new topic to fight for.',
 'Brother you havent changed me.',
 'As many feminists as you guys take treatment from Jake Kabir Singh, you will wake up completely\ntra fine..I like the film, today went to the film for the second time',
 'call dr imo no...008801850322634',
 'I have moved on for thanks for you and need you to go ahead in the lives of people\nfor bad ne',
 'Bhoot gives good review brother',
 'The definition of a man self-sufficient in himself is perfect! 👍',
 'Thats why they say dont comment without watching the full video',
 'Where are you from Monu',
 'How well does he defend Rajdeep',
 'Brother right sides cheek is coming down, she looks like this again in the video\nraha hey tell me pls',
 'Achi movie nahi wahi gheesi stoe waste of time n money...first half a little\nthik thk second half nonsense...lg rahe force koi sit diya ho'
 'Our country says that there is no difference between a girl or a girl, then why is the law\n order my difference, in which a lot of innocents also lose their lives\n even in todays time they have to talk again. nRiste Daro also see 498A LGA, they want to remove it',
 'Thats what I saw. Worth seeing. Ayushmann Khurrana therefore has great respect\nkarti.',
 '@Meaning. A. Hey brother, all of them are late for their fate or for the loss of others\n success will be when it is brought for universal welfare',
 'Neha ji, I think there is no father brother at your house',
 'Shi bat ke re hai aap',
 'Sir Peoples Dress',
 'Someone to keep me with you for the rest of my life',
 'Do you miss Damini?',
 'War Movie Review...',
 'Giska man can live with kera bo kesi also cha bo man or woman',
 "Pratik @sir, Fantastic movie J. Last night's show dek aaya hoon. Nind ud \ngyi bhai, after watching your review he planned the movie. 10/10 star nd\nThis is really epic and fantastic movie. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . Wow",
 'Absolutely true.. bro. \nBollywood has been making us a chutiya for so many years..\nIndia is in trouble. Brothers... save the world..\nBollywood garbage h',
 'Told him right',
 'Whats the matter Dada',
 'Bhot Sahi Bhi I Love You',
 'Why put telegram m post bandh ..',
 'In case of threat to Akhil Khan, \n in Saluja Dimakh. \nCleanser with ridges.',
 '@madhusmita pradhan man try to understand.... Shwetabh bhai has no intentions\n...\n\nAnd the streets are to chalta hai yaar...\nthe streets are not there....emotions are there.. . . . . . . . . . . . . . . . . . . . . . . . . . . . Streets...',
 'Also you are able to use the mic, please tell' 
 'Best whenever you come, you upload the video...',
 'Its right brother, no one\n said anything to the movie Lipstick Under My Burkha',
 '2 Days Me 4 Videos. ',
 'In Chandigarh, a drug addicted girl herself beat boys by driving the wrong car..\nIs this women empowerment.. please make video on Chandigarh case',
 'Jai Siyaram 🙏🙏',
 'The time is gone when there used to be an orte, now the time has changed',
 'Si hei last em chinchilla',
 'Pratik Bhai... Please review\nkijiye of Ajay Devgan sirs The Legend of Bhagat Singh too.'
 'Ha the Lallantop Wala Hai Liberal',
 'soon',
 'Today brother is in form..3 videos in a day..',
 'Matlab will end?',
 'Ek Number Bhai... Great One',
 'Brother you are on point, you must be logged in with this Bollywood joke\nH..Abhi 10 din pehle ki baat hai ek psycho chutye ne ko tik tok bhi apna\nRandi Rona Karta 2 Lad Rona Karta 2 Diya or PHR BC Himself too\nMAR Gya Suar or my city was from Bijnor or he was deeply\ninfluenced by brother-in-law Kabir Singh!!!',
 'You are right brother..',
 'Okay brother'
 'Yaar has been kept in the room, this young man has been kept in his own name, Ranga,The stomach is bloating with laughter',
 'According to me, homosexuals or other sections of the society will only take avi\nbkt to become a mutual harmony ... isli army m they should get permission avi to ni ni ... because it would have been nice to have a poet\nv change but his smudge m or What was the effect on the country\nh ignore this bat it can be done'
 'It was a great review, shutting all your librel and feminists mouth\ndia .... 1 lakh likes for you from my wire...',
 'The boy saw \nfinished brother. ... fell in love ....😂😂😂',
 'This man can save the generation',
 'Police should not arrest in the case of lies',
 'Meet you Sandeep Reddy Vanga',
 'You increase the length of the video by discussing more nonsense than the review... and yes not everyone can think like you.',
 'I saw Kabir Singh or there was only one fear in the whole film. Kahin\nFilm Khatam Na Ho Jaye OSSSSSSM Movie Shahid Kapoor is one of the best actor in Bollywood.'
 'This will be a spoiler review saying false rape case',
 'Why are you doing politics brother',
 'So what are you called, brother, now the Indian Army has formed an army of people?',
 'Brother.. Amazing',
 'Brother Joker Review \n..',
 'Nice move he',
 'Kabir singh chutiya movie',
 'Fun Gaya Reality Jankari',
 'Emotional intelligence has been seen a lot which is a good thing',
 'Kabir does not have sex with another girl after breakup...',
 'It would be nice if you looked at Newton, Tumbbad and the photo like this.',
 '@Seedha openly declared from UP76',
 'There is someone who speaks like this in India. Arundhati Roy Good ️',
 'Respect your feminist',
 'Very good film. section 375',
 'Ayushmann is alive',
 'Nice video sir, it will take 50 years for me to become common in India, then make one our video',
 'I did not notice, brother did not mind',
 'This guy is great'
 'I have just come to see...',
 'Wait more years, brother.. I will do my own film.. he will see how my\nchutiya has been cut and how will he see what happened on the move...\nthis story is different from Bollywood...',
 'Roys trying to understand the way Amit Modi is joking about MTLB H',
 'My movie watching plan is based on your movie review..When you suggest\nWho watch this movie, I also go to see the movie.Thanks'
  'As long as its fake'
 'The girl will have at most 10 lines, brother and 8 were in the ending',
 'If she were a Muslim, Preeti would have had a lot of problems in getting a divorce, first\ntriple divorce was her bad then halala...\nand so too Kabir then not only Sharabs father,\nI have heard that Sharab is haram! handjob'
 'I will watch the film once and then tell you',
 'Corrected',
 '@ Prateek Borade Army people also come from this society, not from any other planet \ Relationships are also good. put extra effort to skill',
 'Amit Shah and (4:6) After the halo was generated. \n \n ** I am not scared,** \n ** I will tell !!**',
 'I love the string of the knot, brother',
 'Speak right boss. ..Mann Ki Baat boldy tune',
 'You slap a lurkey, do FIR rape and show in front of the class that this is my captive, what is this',
 'Kutte loves to travel',
 'It is wrong',
 'Earlier I too agreed with the point of some feminists, but after your review, my views about Kabir Singh have completely changed. The acting was undoubtedly already\nliked. Now I feel like seeing 1 time or more',
 'why use dirty words... please stop it... disturbs me',
 'Divorce should also be made easy. If two people cannot live with each other, then for the sake of getting instant divorce',
 "My India's Sons Trailer Review - \n<https://youtu.be/DSILsKSeM3g> \nPink Movie Review - <https://youtu.be/EiVSu8MPV7M>",
 'Brother, dont drink water in a plastic bottle.',
 'Then will see that false cases will stop happening\n the house will stop breaking',
 '@crazy Ram you dont give your whatsapp',
 'Everyone is talking about Preeti or Kabir, no one is talking about Shiva, to whom\nKabir took care of:, I want a friend like Shiva in real life, who stays with me at every turn. If you agree then like crow....',
 'so late',
 'Truth is bitter...',
 'Hahahahaha',
 'Amazing brother, this was a real review. You also\nopen a movie review channel, you will become the best critic.',
 'No need to give stars.. write full',
 '@Afrid zmn commenter',
 'There was a support, that too went. No wife, no gf.',
 'Missing, and the lady, how are you?',
 'Fai Yeh Arun Film Ka Hai',
 'It is fun brother seeing your review',
 'You have accomplished do something new',
 'нιнιнι only нтee rιendнιp regι ',
 'O woman, you will do well',
 'Rupesh rothor bhai review dashari video mw h ye...reply to liberal j',
 'Leave theory, think practically!',
 'Ek Number Bawa',
 'Pratik Teku now modern cinema seems like a jigsaw.. earlier bhot buddhe tell\nkarta was you, well, keep it up!',
 'A lot will be good',
 'Bhai Teri Bhi Toh Wing is going to know',
 'Its a very good movie, Bhai Shahids acting Jabra',
 'Kitna Hot Top Boy J Kas Mera B.F Hita V',
 'Mr. Goat... speak slow....',
 'I havent seen it'
 'Brother, I have seen 2 cigarettes vs.',
 'Brother he is a great critics'
 'Bhai Kabir Singh watched and made a video',
 'Ow stree...😀😀😀😀😀😀😀 \nwill you make friends...😀😀😀😀😀',
 'Saurabh you are well maintained, you are predicted to live for an hour.',
 'Best courtdrama film Ive seen till date, 10 out of 10',
 'Brother, you tell this'
  'Bhai does review on Joker...',
 'I Wanted To Have A Video',
 'Bhai spoiler alert dal dete',
 'Plus such a vote solution, I have no sympathy for Hindus in my heart,\nMany of my friends are Hindu and I respect all of them very much.',
 "Hey brother, you are absolutely right but she is doing backwas from 'Not a Movie' Didi",
 'fathers son',
 '@Fact Park will give a video of some tick to the ad,\nyea people will be happy in it',
 'Till Hindustan there are sunima...',
 'If there is an Indian, then the star of the Indian is to be loved, brother. There is some concept\n of India which is real, otherwise the word is being copied in the country.'
 'Hmmmm.. you look like a devotee',
 'Dhansu Review',
 'They are strong, they are also in Smaglita',
 'I liked Kabir Singh for acting only from Shahid Kapoor Bakki to Bollywood to\nChhutiya Tha Aur Rahega...'
 '25 lakh gays in India, sir how did this data come out',
 'Review done very quickly, brother does it comfortably..',
 'Top Movie Hai Prateek Bhai',
 'And laundo whats going on',
 'Bhai Saheb Shahid Kapoors violent look and his violent character were liked by the people... the rest of the story is a baker.',
 'Billallallalkul sahi baat j..bhai love you',
 'Mood off?',
 'Yeh librels hai, who has eaten from where these people',
 'Hmm thanks sir'
 'Mumbai Chakla',
 'I only watch the review of 2 Jaane, one Komal Nahata ji and one your Nour before watching your review, I saw Komal Nahata jis film review\nI am going to watch tomorrow',
 'Bhai mere dil ki baat boldy tune',
 'The side effects that happen after watching Bollywoods garbage movies are its script\nHey Kabir Singh... Relatable thats why everyone else is because watching Bollywood\nk we are like that.'
 'Brother Breaking Bad review',
 'Rightly said brother',
 'BJP got new inverter money???',
 'Sir keval movie review karo saal..',
 '@PRATIK SAMAL hn dekhichi',
 'Perfect brother'
 'If such a big platform of the country accepts homosexuals, then the country will learn\nko v, that a persons personal life or professional life is different. Everyone should get a chance in one sector.',
 'What is the process of 10 years census',
 'Liberism is a threat to the whole world',
 'No that movie was thick',
 'Brothers point is right, but one is beautiful too. It is okay not to be dangerous, but the remedy\n is being done.',
 'Got to see a good film in Bollywood after a long time and I saw a Bollywood movie Bahut Din. nice video. Review Kaafir (Zee5)'
 '@KoshalSharma you are giving proof of wrong thing. Give it to someone who says that there is no stone pelting in Kashmir. I didnt say that. Then speaking, properly\nread what I wrote. Then the society will come.',
 'Brother love you\nyou changed the way I look at life',
 'One no. brother',
 'Now what will happen to the people of Tik-Tok????',
 'Speak Hindi',
 'Happy brother get good reviews then he will tell...',
 'I wish my country is even 50% of your maturity, then where did the country reach\njaye,',
 'You make a mistake',
 'sir telegram channel py aapk video k pdf kue nhi upload kiyeh jatay',
 'What is the mindset of these dislikers',
 'Kitami Backchodi',
 'has done it'  
'Simply made a law that if there is evidence that if the wifes keys\nallegations are proved wrong, Usman Kitna Kitna Hi Time Lacey... ..',
 'Jio Boss Jio',
 'Aa Bhi Jaa Sanam,\nYun Na Kar Sitam,\nYour Beh###',
 'Tashrifed broke broke also screamed, that today is in the paikhana! \n more',
 'Is this terrified you have never seen the power of Kabir Singh',
 'There is a review on the trailer of Bhai Dabangg 3',
 'Brother, how soon I have edited the video of the video looking at the left bottom',
 'Yeah the point is perfect....the effect of the film is a lot.....my friend\nhas put his fb id and pub name kabir singh + photo v him ka hai\n😂😂😂',
 'If there is talk of politics in the name of such review, then I write that 1M will never belong to this channel. Subscriber\nBadhenge.👍 if you give review very soon',
 'Tell the truth whether you had to prepare the speech',
 'While watching a movie, it would be good to read the points of both section 375 or 376\nky... \n \nI have been taught',
 'AO NA',
 'There was a time when it was the brother who made Arjuna ready suggestion',
 '☺ now felt that India is changing the arrears',
 'I want a beautiful gay husband',
 'Yes yes yes... that is the best part of the video.\nNo feminist will say anything on it now\nPratik Bhai Rocks',
 '@vinita pant 7681831967',
 'I havent even seen Bhenchhod',
 'One dead fish has the power to make the whole pond dirty... thats all I think sir..,',
 'Yaar Chor Do make such films, make films based on real events or reality\nmake films based on stories',
 'Shi say',
 'Good morning daughter',
 'You gave the right answer...',
 'A dam brother',
 'Very poor introduction'
 'First of all find out who are Liberals... Please dont bring religion in the middle!',
 'Bhai Joker Movie Review',
 'When is your book coming? eagerly waiting',
 'It was funny brother',
 'Allah Hidayat Di Amen',
 'The poors land would have been a baker',
 '@osm bro bhai you are fluttering so much, amja agaya......a little more',
 'Kashi Ram Nishad Kashi Ram Nishad 8140358886',
 'You find some other work...',
 'ha wow',
 'Amazing sir, feminist irritating work is needless to talk',
 'Or please tell us that your book is KB Aari... I am waiting for your books ghost\nBhaiya...',
 '[13:08](https://www.youtube.com/watch?v=N_ZMfQMZos0&t=13m08s) Little Control\nkaro',
 'Shahid Khan',
 "Oh launde kya hal challine is missing",
 'Ta Modiji or Saha didnt lie?',
 '@Ramchandra Chhatar Han',
 'Krishna should not even give permission....',
 'Pose only with the bat or play lying down',
 '@ Avinash No. Deveskar to her AK Quashan Key',
 'I DONT WATCH BOLLYWOOD MOVE TOTAL GARBAGE H!!!!!!!!!!!!!!!',
 'I closed for half an hour',
 'Looking at you, I started thinking like you, Bucky thinks that Faltu is being cut from \nduniyA but he knows that he is right... Thanak',
 'Who is watching like this in Hindi',
 'ap ap your video kare me sammilit english subtitles. There will be light in your Muluyavan speech\npure brahmand. Bharat Varsha will become the world guru of a bright young soul like AP. Thank you Shrey. I will have full faith in you\nhai ki ap English subtitles.'
'Bhaijaan, you are my brother, I believe in eye for eye, you have 4 lashes \nbitch ha, if you believe, then you have to end the trouble, once that case is brought back, I will leave his\n mothers mother, First of all, I will leave my mother-in-law who has done the case, my father has gone mad because of him!!!',
 '@KishanMishra if the character talks so much, then why did people like Salman Khan\ns character named after you so much.',
 'Mubarak reviews are better',
 'I am seeing so much patience Arnab for the first time.',
 'Brother plz make a video about Tiktok too. Like friends for tiktok\nchutiapa roast',
 'Jai Nakra in four more shots?',
 'The man who will say tha tha khatam kar karoonga... I was feeling that Dr.\nsomewhere...not only the people of Sherya and the Timeliner should "finish"... ',
 'Brother... I havent even seen the movie "Kabir Singh"... Knew it must have been loaded with chutiyapa... .......',
 'Bru Salil Jamdars Bhi Item Song Parody J...',
 'Brother, you are right, but what the audience wants will be seen, wont Bollywood?\nWhen will the taste of the audience change?',
 'After comparing India with America, this point comes that. \nThe trends or problems that are going on in India were going on in America around 1940-1950...',
 'You are a chutiya, submit the paper coming to NRC.',
 '@Gaurika Fry by Ritwik these are all nonsense',
 'Hey Dada \n🙄😄🤩kitni mistake \nSahid Khan....Kabir Khan.....Kabir Singh',
 'Bilkul shi bol re ho sir, let there be someone who is speaking against people\nhai....👏👏👏👏👏👏',
 'exquisite',
 'with great',
 '@Lolite why son is your mother calling, am I free to meet',
 'Yeh saali chutiya movie kabir singh not seen yet, but my friend pagal ho\nrkhe for its song',
 'Group hypnosis going on',
 'Arundhati ️☠️☠️☠️☠️',
 "They don't know a man's feeling. Real me a boy. When true love betrayed me, these feelings ate his auras. Sandeep Bhangra knows the inner\n feeling of a man. And those feelings that khut Khoot ke bhariye movie Nandar..every single scene is so realistic that what I tell..",
 'Very good brother...\nThank you...',
 'Whi to this day even a mystery H',
 'Where are we suffering right today',
 'Bhai Bhat Masti',
 'The third son of Shashi Tarur!',
 'Giving reviews by watching online for free',
 '[7:26](https://www.youtube.com/watch?v=N_ZMfQMZos0&t=7m26s) The truth told\nhai...😂😂😂',
 'You are right brother, you stole my face',
 'look at bangla sentence',
 'Jahangir Bhai did a wonderful job',
 '@Ching pong bhai my darling... 2 years +2 BS... I have a relationship of 7 years...\n25 age... experience age is more than you... good thing if life time together\ nrhe..... or god forbid if something will happen if breakup happens... then remember the name Rahul\nSHARMA... go ahead krishna bhat is difficult...',
 '**Administrative Committee✊🚩** \n \n😡😡😡😡😡😡😡😡😡 \n **Player Featuring Aryo Kokima Who Wants to Make Huru Qureshi (Jihadi)\n, and Conducted The Hindu Impossible There are members of the name Varna in the smoke of the Hindu.\n \n **This is associated with Arya Out and the Aircraft is broadcast. , by eating cow meat\n were failed in the country** \n \n**😡 Some units are members of Kuri wrong\n \n \n \n \n \n \n \n \n \ n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n *😡circumcision, separation of three, halala, mutaah, halad and the deadly disease of the innocent, burkha, and\ninnumerable evils filled the attention of superpara Injihad\njad** \n< https://youtu.be/1HfIlxdxRhE> \n \ n**after all, how long Hindus tolerate, for how long do they bear the demerits of their religion... and\nwhy🤔🤔😔** \n \n **🤺Wake Hind is necessary to destroy the germs that protect their religion When the insect is protected \n\n\n \n **The germ is destroyed in the complete exterior of the warriors** \n \n[https://m.facebook.com/story.php?story_fbid=3329309197094463&id= 2262905757068151](https://m.facebook.com/story.php?story_fbid=3329309197094463&id=2262905757068151) \n \n 🙏🚩🇮🇳🔱🏹🐚🕉',
 'When do you meet such people',
 'Brother, you also become a director, Shyam, a good film will come in the market',
 'Dream girl ka karo pls',
 'Babhut relaxed me [0:36](https://www.youtube.com/watch?v=J2J5ssSP5yQ&t=0m36s) \nSavage😎',
 'Thats right guru..',
 '# hat teri... ooh',
 'Say in Gujarat',
 "I have seen so many videos of this but still I don't understand why girl\nbanke?"
 'Nice video dude',
 'correct',
 'Is it right to be conservative if I dont become a Liberal??'
 'Why brother gay is not allowed',
 'Freedom to speak is when you are speaking right...\nRajdeep is also an unofficial member of Congress party.'
'Good review but vedio kj tenth km rakha kro yaar q kj 16 watching time of 17 minutes\ndil ​​nhi hota vedio pr click krne kq',
 'Oh friendship will you osam',
 'right brother'
 'said',
 'Zurroor denge brother',
 'Its true that I am English quite a lot ae film but its film is very good and films based on this kind of material are needed and i hope in the end i fail right\nbut best ae',
 '@HeartHacker theres more......',
 '[1:20](https://www.youtube.com/watch?v=N_ZMfQMZos0&t=1m20s) \nThe sound of the chair is brother ne pada naya mara',
 'Now if we do, you will get Burma',
 'Film To Kal Aa Rahi Yaar',
 'Bhai Web Series M Sirf Gali Aur Sex Hota Ja Vedio Banao Big Brother Youth Ko\nsmjhao',
 'A1 Movie Hai Sir',
 'Shi bol re ho bhai',
 'It is absolutely right sir, we all support you',
 'Simple si baat hai... because in those films I glorified\n drink, smoke in baton... like Veere Di Wedding, Four More Shots Please etc. \nhai looked like a bad phase! And ending me Kabirs father also says to give up the drink!',
 'Brother I only see your review',
 'Critics dont know about this movie bare',
 'I found this movie absolutely rubbish, didnt even watch the whole..',
 'Break the chains, brother, you have spoken your mind',
 'There are still these people in the army.. my friends have told',
 'Ha buhut acchi lagi... very comedy thi ending very nice',
 'Ranga Badla',
 'Lol Ranu Mandali',
 'When will you review the film',
 'He is talking about movie talk',
 'It felt like',
 'Better than this is Mobark sir',
 'Bhai sahab subscribed because of t-shirt, the same t-shirt is l\nsize..[wyo.in](http://wyo.in/)',
 'Very good... I found B Kabir Singhs film absolutely lousy',
 'You film. Happy to see brother, after 2 hours of film, now ji and the policy will be clear, this is really a very good film, you look back Bollywoods\n**unique** story',
 'I was wondering why Kabir Singh became so popular...',
 '<https://youtu.be/47uX4qwlVnQ> \n \nLook what the BBC one is saying!!!',
 'Review of Apna Bhai Ab De Raha J Kabir Singh Jab Yeh Movie Max Pe Aati Hain with Sooryavansham. \nBrother, do one thing and post the review of Suryavansham tomorrow.',
 'Yeh sb toh thik h ...... bt how will you identify the gay',
 'Found a lot of relief Got a lot of relief',
 'Hey bhai bhai bhai bhai 4 videos in 2 days',
 'This Rajdeep will never improve',
 'Have you not shaved properly, brother... one side is trimmed and one side is not',
 'You are right, Sab Faltu J',
 'Not seen yet\n[#kabirsingh](http://www.youtube.com/results?search_query=%23kabirsingh)',
 'Truth is bitter!!',
 'Brother reply \nwhat other job do you do after review\nhow do you watch all the movies',
 'plus arjun reddy movie link',
 'Shobhit Kumar, whats wrong with this?',
 'Oh hoo symbol brother, you have told a lie',
 'I am a tomboy or the one who knows my nonsense masterpiece Krita Ray 377 act new gay or\nlesbian person ko jaan di hi',
 'Sabit what do you want to do from the film...',
 'Long live Rama',
 'Omg my god \n\n all the more congrats',
 'Sir, you always talk about the girl, do you have any \ngirl hawk? j'
'Ranu ka number 9884057000',
 '[0:40](https://www.youtube.com/watch?v=J2J5ssSP5yQ&t=0m40s) Really who\nrelieved no.......',
 'I think now fear and live in religion people should make agreement paper\nchaye in which it would be written that we were having sex of our own free will, it would be such that the girl is not a victim of rape on the girl,\nkyu, what is the highest in the country It is big,,,\nkyu every man should have sex Get,,,,\nPlease make a video on the idea,,,',
 'Brother, I also told my friends that there is nothing, the same story is seen again and again with a new face, but my free friends went crazy after watching the film',
 'Another amazing review.... Aaj ki saw film',
 'He: Your glasses are very good \nCarwala: wrote my wife',
 'O enemy will you do friendship',
 'Brother, what happened to your beard (Dadhi)'
 'Heard brother talked',
 'Jai Mata ji ki sa',
 'Critics mafia is giving work stars..Mathlab movie is definitely good for country and\nLogan..',
 'We the girl had called Kabir herself at home... Didnt watch the film properly??\nI didnt understand the film??',
 'What will happen to you Sinab Ali?',
 'Ashish sir, Ashish sir with Guar Larki Aisi Nahi Hota. Was she straight or naive?'
 'I said that this guy is better than Sandeep Meshwari',
 'One clap to your thinking',
 'Guys please watch this video. Right wrong maybe understand \n<https://youtu.be/nNekk07Qt2g>',
 'Harmi ho gaye kuch pade like those who took for Indian culture, said J.. Kabir Singh\ nm nothing like this glut .. Bhoot sandar for the one who loves J.. Bhi pyar kiya \ nhi if he will say it only'
 'Dont use the bat of the baker, dont speak naked in the picture,\nstop making long videos\npls',
 'The hill line of the movie creates interest... an instant connection is made\n... Fantastic movie...',
 '200 is done',
 'Brother someone tells me that background music Kabir Singh when his brothers\nBachelor enters the party TB Ek DJ music type Laga Tha Mast Tha.'
 'Not Saurabh Shukla',
 'Will you put the video of the year today itself, so much work Uffi',
 'Any Bollywood review which is valid',
 'I write my gf r female friends ke kana aadhe bando n saw this movie\nh...',
 'BJP is there',
 'Brother Hindi Mein Tiny time is more. Thats it, nothing else.',
 'Bhai Lakshya Chaudhary took his equal in he \nwatch that video once',
 'So dont you see, we have to watch videos like this, stay engaged Prateek Bhai',
 'Joker please'
 'Long live Rama',
 'Dont love like Kuttan?',
 'sourav ji you are very psnd to me..whatever you say o psnd h..or i speak v to my\nfrnd o listen to you...mgr i left this because of netanagari\nh ..bcz rajdeep jo The bat wearer h ... clearly visible h..1 side bare me only bat\nkrta h ...he once called v Anubhuti Roy a bad nee???',
 'Akshay Khanna is a great actor',
 'Swetabh Bhai was a great review',
 'Live......',
 'Bhai app youtubers Allah app what works?',
 'O: 04 Today we will talk about Shahid Khan movie Nade Mere..\nShahid Khan.? New Kabir Khan? New Kabir Singh, Shahid Kapoor what are you saying\nI am.! ',
 'This is Gods year',
 'Sir Nice to hear your review...',
 "Nani's \ngang leader do brother. Please",
 'Sir please titanic movie review kiji request h please',
 'Arundhati Roy is absolutely right... boycotting NRC NPR',
 '319 dislikes.. who are these people...',
 'See also Shutter Island brother',
 'Aap pasand do a film brother',
 'Movies number one',
 'Yaar Punjabi Chha Ve Banaya Karo Video',
 'Soon Ill be making a video on it, a little less left'
'Brother, after watching the movie, it was the same RXN that if someone does all this in real life then\nhe is a no. It will take true love and not true love',
 'Bhai Shaadi Mein definitely come review the film please',
 '+Ajay Kumar Singh ha ha true...',
 'Great movie'
 'Sir become CGPSCs current affairs',
 'Jai Shri Ram',
 'live brother'
 'You dont watch movies',
 'You should make your review videos of such short length',
 'They take it only after physically checking they are not gay people',
 'Useless...',
 'Right, if we have to move on with the world. To become the worlds track\n law. Need physical strength or patriotism. Personal life should not be kept SE\nMATLAB.',
 'Brother, your words enter your mind and make you think.',
 'Hi do you show the video',
 'Jio Boss',
 'Brother, I am a member of the app censor board, do not give knowledge, review',
 'Link to watch Kabir Singh movie <https://youtu.be/i9QCYiTlBmU>',
 'Pallavi Shahdev Hmm',
 '@ Shubham Bodade and you become bald after seeing Kabir Singh',
 'V wanted to wrap the judge',
 'Kabir.. as I had imagined already...\nIt seemed that brother will speak something on Kabir Singh',
 'I havent seen the film...but seeing the review, I havent seen it',
 ' @ Kamal Ranjan brother, I also eat that "not in a public way" \nhey',
 'Sir to review the poster of Bigil too',
 'Baker what movie?',
 'The main point is that .... Kabir Singh is a lively person with anger...\n... when he is full of anger, then the woman looks at something...',
 '[8:07](https://www.youtube.com/watch?v=N_ZMfQMZos0&t=8m07s) Girl came boy\nko love falls... \nHow does love happen brother? \nThen the same old formula of Bollywood..\nthe boy saw the girl\nthe end thing is that love has become so much that he will kill for her',
 'Jasleen Kaur was made on her',
 'If there is a situation from the next debate, then how is this situation',
 'Brother there are different size video watching pads...recommendations\nhi of your video coming from youtube...all buttons are pressed',
 'Who are these liberals \nwhere do these people come from',
 'The bad of watching a movie and its bad symbol looks like a review of brother like\nSoha pe suhaga.',
 'Brother, if there was a love wala or 1k like option, then the video became a lie,\nthank you brother. It was much needed \n🙏 🙏 🙏 \nIt was very much needed to shut its mouth',
 "Come on man, it wasn't as bad as this guy is telling.\n I think his ego is hurt, how did such a good movie\n'Bollywood' turn out to be."
 'Great brother'
 'O woman, will you make friends?',
 'Pratik Bhai review the movie and make a video to talk in its details please',
 'Who makes Bollywood Bajirao Peshwas Devdas type character who goes mad in love and die...',
 'O woman will you befriend???',
 'Got right',
 'Plus give me sir. I am a person of my job search search. I have lost 2 lakh rupees with \n good fraud',
 'Sir titanic movie review please',
 'How does love fall in love by looking beautiful, man I still do not understand...\nhumans are so stupid man',
 '@DeepakGoel maybe Bhagavad Gita Mai Ho Answer',
 'Puri Movie Hi Dikha Do... Yehi Pe',
 'You are real! \nI am saying emotional or not...whatever you have said till today, oh true hue came\nhein.',
 'V kaya diya vii',
 'Mast film j bhai, maza aa gaya dekh ke film...',
 'So what should be the rating from your point of view' 
'This video is right he didnt join shide army on this he should start starting for little\nniche levels like police or someone or field\nme or had joined army bad to see their work Do or not think of \nchahiye q ki article 14 se 32 ke bich unko bhi adhikar he',
 'Brother, he has not got his status yet',
 'Sab thick hi sir..but app review more than dussere topic ke nade mujhe bol rahe hai...this app can v in your other channel....but app review every\naspects of the movie. ...',
 'Namaste Sir,\nThe Constitutions right to symmetry has been amended and this is to ensure that it is correct to be correct. \nAnd those who changed the rules 5 and 4 6 pixels of the force and the force of the army\nthe message of the parliament will still talk about our army chief Bipin Rao Samb\nThere is no disregard for the constitution.'
 'Sir you said a dam thick.. these people did not understand the flim',
 'Top of MTLB',
 'Brother you told the truth about your or my views about akdm movies, the same h akdm',
 'Fear is the only thought.'
 'Jinka Jinka is still a breakup and those people dont if you see this film, you smudge\nLoad laga gaya of those bands...',
 'Love to Love Hai Boss',
 'Its okay to say brother, Ranu Bhai is the baker',
 'Kamini Halkat in Innova Faridabad tomorrow',
 'Looks like your Hindi is weak... Watched the video again, maybe this time you\nSociety get our video 🙏',
 '@ Pratik Borade Drinking or smoking cigarette is not a bad thing, but raising a hat on a girl\n is a bad thing that too without victory',
 'I havent even seen the movie. \nThe ringtone of the movie is just right😂😂\nVada Vada Vada Vada Vada Vada Vada Pav',
 'Abhijeet Desai Tu Bhai Jaa Na movie, why did you come to see other review?',
 'Hey brother brother brother brother, how do people change after money',
 'I got blacked out while watching the video',
 'Somebody would have kissed your sister in college without her permission....so \ntereko will do?',
 'Brother I also felt like bhot baker... but I would feel that you came to \njustify the movie... because everyone was appreciating it...',
 'Not everyone is the same'
 'And what are you doing,'
 'Hey whats happening... CCC',
 'Homosexuals are not brave.',
 'Great brother'
 'I laughed the most when the character Kabir Singhs friend Shiv Kabir\nko offers him to marry his sister. BC such a man how can anyone give his sister\n.',
 'Join the gym to become a Carlo Kuz body',
 'Bohat hard brother',
 'He has given spoiler, he tells this brother...',
 'Bhatt nice gay lesbian would have any pap nehi gaye to marry a larki ki\nlife wasted kana a pap he ur lesbian to marry a boys life wasted\nkarna pap he .. life apni he apne tariq se khul ke jio ghut ghut K no..',
 'Mujhse Bhi Jhand Film Lagi Bhai',
 'Like Pratik Bhai from Indias resettable video for India,',
 'Hahahahahaha..the industry with the path of owls Bollywood Kya Sach\nMast Baat Kardi Bhai'
 'Bhai BS ek galti kar di tune...\nthis video was very fruity',
 '😖😞 Sir, even in my family, the wife threatened to commit suicide to her husband, threatened to recover 25 lakhs, he would have threatened to get caught in 498a, ........... Sir my request is that if 498a Misuse of or other dowry case Per\n, the wrong use shall be punished with imprisonment of 1 month to 1 year or fine of 1 to 5 lakhs\nchahiye or the lawyer should be directed that the next time the ESA happens, we\npenalize Give hog. ',
 'My life has improved a bit... so only because of Shwetabh...',
 'Homosex the DBA and serve the country',
 '@aggressive squad hain i have seen us',
 'Brother but didnt you say anything about the performance of the actors?',
 'Alexander khaki hand came and khaki hand went',
 'If you speak, brother is good, how much do you speak brother',
 'Your right said symbol Vai',
 'Kabir Singh is a top class film, entertainment at its best. I felt victory more impressive than that martyr.',
 'Or all is well...brother T-shirt',
 'Aarey didi didnt even see the trailer',
 'Eat absolutely right Shwetab Bhai',
 'Thats his old habit..',
 'Long live Rama',
 '@Hrdk Parmar Ha Dekha Yaara',
 'Brother, what is this Tiktok..'
'Reflection... because there was no light!',
 'I heard you said I am...',
 'Rajdeeps double standards...Afraid to criticize Arundhati,\nwill speak anything to PM',
 'If you believe my brother, I have seen this only yesterday',
 'Brother perfect review',
 'Brother, you work on the left more than on the right...',
 'Bhai Sari is a masterpiece not a film',
 '_Kis Kis Ko Jokers movie review?_',
 'Pratik Bhai...Pink Movie V Apni Jaga Sahi Hai...It is not that what has been tried to be seen in that\nmovie...it is totally wrong...',
 'Okay said brother'
 'Brother has not made friends till now, has any woman',
 'Right bro only brother saved me from being destroyed by a fraud girl',
 'What does the society want in brother \nhahahaha',
 'Hail Siya Ram',
 "Sex and love are two completely different things.. Sometimes I feel very bad for boys and men.. Always ladka hi wrong ne hota hai..",
 '@SHAYRI & WHATSAPP STATUS are bhai who is natural gay, na o different thing\nbut jo achcha khasa ladka ho ke we gaye ho gaye hai ye glt hai',
 'The more fun I got after watching the film, the more I enjoyed watching your review',
 'You are right...',
 'Review in Hindi',
 'Bhat tough',
 'Love you Shwetabh... there is a reality on your every thing...',
 'Aag Laga Re Bhai in your community from today 3 videos todays jumma (Friday)\nka kiss* from Shwetabh bhai....',
 'Rightly said',
 'Shahid Khan Gangs Of Wasseypur Wala J',
 'I havent seen a single Bollywood movie for 10 years,\nhu only watched a Hollywood movie. I am 20 years old now',
 'Joe army wala/wali married h his wife/husband is having problem ji\njo gher pr h\nBCJ army camp in sb some get rahe h\nThink border pr 2 foji in "bunker"',
 'Brother 2 ad is shown, then brother will get support..so if the advertisement comes,\nlooked crow',
 'Man, I keep watching your ball, its cool brother.',
 'Dude Timeliners TM logos video me in a group Lady Hulk was sitting with her \ nkch puchte zara',
 'Correct the jagir brother',
 'Talking about Not a Movie Review',
 'This is in front of the critics.'
 'All those who are involved in the 498A case should see these vedios',
 '@shi..! Really my brother.... Shwetabh bhai had told in a video that no world\nI can ignore everything but any new film come new celebrity\nkids, their public relations teams come out from there. v\nneed to watch the trailer',
 'Ive seen it all. You must have seen that much because\ninterest in everything else has decreased',
 'You are a great fan of people',
 'Brother rightly said .... Yeh India and aju baju like we are in Bangladesh',
 'Vocabulary is fine Bhadve',
 'Thequint Vavalo reviewed his game. \n<https://hindi.thequint.com/voices/opinion/shahid-kapoor-kabir-singh-vijay-\ndeverakonda-arjun-reddy-promotes-toxic-masculinity>',
 'Wife had brought hahahahaha',
 'Vai Sabbas... 200% true.',
 'Brother m love your video ko kates journey',
 'We were thinking the same',
 'Very nice video'
 "Have seen the movie 'Bridge to Terabithia'? Otherwise watch it..",
 '[8:32](https://www.youtube.com/watch?v=N_ZMfQMZos0&t=8m32s) Brother has started promoting \nTupperware too',
 '@BhaveshSanadia Top MTLB',
 'Fantastic Chandu Bhaiya.....2003 ..../',
 'Its difficult at first...but even after being selected\nhai',
 'Made it a case STD in classes' 
  'Perfect Year',
 'May we agree with your words .. asi film oo se se chheye we all should \ nkaha not make mistakes .. so thank you so much sir ashi hi good .. video ki \ napeksha karti hu',
 'Sai Kha',
 'What the hell is this',
 'Its right brother \nbit hard',
 'Ranu Di Jio',
 'Shwetabha bhai ka yai to mast lagna h way of presentation',
 '@ManishGoplani What did you like us about in the movie?',
 'I havent seen the movie but.. lets get to learn something',
 'Shwetab bhai\n review on the movie Joker 🃏 plssss',
 'Kabirs film is 6/10',
 '**Kabir Singh movie at all**** not related to practical life**',
 'Brother, please do a full review of Stranger Things',
 'Lust Stories etc. I dont watch sexual movies or watch porn instead\nCarlo Judge',
 'She has a life of her own',
 'Kabir Singh Chutiya Film H',
 'Bengali Movie Durgeshgorer Guptodhan Ki Movie Review... \n"Guptodhaner Sondhane" is available on Prime Video.... Movie Review Do Iska',
 'Boohot good spoken\nvai....[#loveyoubro](http://www.youtube.com/results?search_query=%23loveyoubro)',
 'Its like that'
 'Yeh Bhaijaan who is speaking is right.. but there is a doubt that if any director or producer of Bollywood offers him a role, he will never say no',
 '@RevatiRamanKumar can be normal movie review too',
 '@AliKhan please answer me',
 'If you want to see **toxic relationship** then watch Nana Patekars film **Agni\nSakshi**',
 'Kesari \nAndhadhun \nBhadhai Ho \nBatla House',
 'Llantop V sold out?',
 'Someone said in the comments, came to see the review of the film Liberal and feminist\nKya j kisa j sun nahi...',
 'Brother keep going like this, I supported you',
 'Brother is right'
 'Kya man did your very bad \nspoiler ky diya man',
 'Jai Shri Ram Prateek Bhai.. 🙏',
 '@sadanandkamble bhai listens to me too.',
 'Give some answer brother or the answer is no. ??',
 'Pratik Bhai, you have explained everything very well, but he\'s problem is not with this film, his problem is with this film being a hit.',
 'Make little videos Prateek sir..',
 'You will be friends'
 'I am with you brother',
 'Brother you watch the anime, see his story',
 '@nripen rajbongshi kiya karoge',
 'Topi Wala Kotor Bala',
 'Get captive in college as seen in the film and pay 5 years...',
 'It was good brother'
 'Or if the girl slaps the boy then Bo Xi according to you',
 'Jazeeb Ali',
 'High quality has given shelter to homosexuals in our country, as long as \nour society blames it from the root,\nwe cant expect all kinds of change:|',
 'Whoever is physically mentally fit h, patriot h, army lover h, wants to serve the country\nkarna by joining the army, he should get\nchance without any discrimination of caste,gender...thank you handjob'
 'Thank God brother I can now my J..',
 'Greetings to you brother Surabji, once you Pushpendra',
 'Brother now make video on coding too',
 '@INDIAS TOP 5 The Bagirath who took the Ganges was given\nthe union of two mothers.',
 'Whi vii do famous kar kya karega on youtube'     
'Lark ghar mein chorke aana bhaag hai while watching a Bollywood movie. \nPlease dont get into arguments',
 'Chutiya Film...',
 'Amrit Anand is dangerous now so join it',
 '@ PankajChouhan abe like you men feel this, so this lesson..\nIt is also necessary for you..',
 'Bro House Of Card Review Karona',
 'Speak right brother gg',
 'Hello my a human advisor hoo my shwetabh gangwar sir bohot kuch shika\nhu.. but shwetabha sir wont be able to make video for every single boy or girl then\ love break-up to nagar ap also ect problem h to number - 6200845455 contact me\n10am to 8pm',
 'Brother, your videos fps is 30... well... keep it up...',
 'Its a painful, harsh, whimsical, carefree, painful, love story',
 'Pratik Bhai Dhodya .... true ..... only decently ... and Bhat only logical\n would have given his answer ... feminists who respect Indian culture.. ... Have to understand…..the place of Mother Durga in India\nthe highest is seen……if you can understand this further.',
 'Wai Maharanis Korean Drama Review Koro',
 '@execute right brother',
 '10/10... ohhhh... now it will have to be seen brother',
 'Negative positive Asad Jassar Negative is that we can judge someone by his/her gender and when he/she will join the army for his country, for the country. Positive Yeh hs Bipin Rawat says in sentence hs army homosexuals are not allowed\n ji armys acts which h 45 and 46A.....',
 'Netflix too has done masters in Faltu thing.',
 'What is said to be accepted by right words...',
 'Thank you Deepika Bharadwaj'
 'Bole you must have been realistic',
 '** Shwetabha Bhaiya, this is a wrong thing, you are \ndaalte a lot of videos at once. Its a good week / 10 days in one video\ndaalo !!**',
 'Review The Darkknight Please',
 'Todays view is better than everyday',
 'Which Jaga Ja Address Give',
 'I did b yahi kiya ....\nnow my life has taken time..',
 'Truth Kadwa Hota Hai is an example of this you can find out by watching the dislike of the video',
 'Bhai Kya Baat Hai Back to Back Video... and Kaisa Hai Lownde',
 'Hindu Muslim riots only harm politics..otherwise people are peaceful',
 'Nice Video Realty Hai Bai',
 'I dont even know how the film took 280 crores.',
 'Yeh short movie fucks...hai...is sabke..thot..kharab aaye gaye LGBT..ke bare..I\nkoi aisi to naye kare ga or..jo..ge.ho...te. .ho they..a goon..nai hote.\ndecent or insent...hote..koi..such a short film..banvo..whose..everything. Banvo.plz...😞😞🙏🙏🙏',
 'You are right'
 'Gay Hind',
 '@MayankTyagi yes',
 'Brother in the thread Channi ku closed?',
 'Aapse koi koi koi karna aapse huge review of any film You should get an Oscar for\nMovie Review. Thanks a lot',
 'Section 375 and Pailwan are both cool movies..',
 'It is also good for the boy that the girl should not be misused.. by looking at her talent\n try to move forward and not by misusing or the girl.. the boy should be decided on a trial\nglt case',
 'Ranga Billa... Arnabh',
 'Shahid Kapoors elder brother will lage you',
 'Gadar Video...Bahut Hai Aa Hai Hai',
 'Pura Movie Tell App',
 'Brother help plz....ma not working hard because of loneliness...ma na\n not able to learn English.. scared of friends because of honey\ i cant do anything in life I will find'
 'Ranga ya billa do zalim tha',
 'I havent seen Kabir Singhs film yet',
 'Forget Ravi ji now the girl is not like this',
 ' Suchitra Tyagi has a lot of problems with the film',
 'The Great Alexandra kept Gay in her army and ruled the world\ntha...',
 'Army Mai leaves after being a little physically unfit...',
 'Auto Wale Bhaiya Best',
 'Is it male or female..',
 'Ummmm... if someone has the potential to serve the country, then connect service to the country with his personal\n life, then he would love the country\n would love the country if someone was ready to die for him Can anyone... to protect the country\nwould think that he is gay or if he wants to protect the country....he lives in the country'




]

In [ ]:
df1 = pd.read_csv('/content/cag100w.csv')
df2 = pd.read_csv('/content/cag200w.csv')
df3 = pd.read_csv('/content/cag300w (1).csv')
df4 = pd.read_csv('/content/cag50w.csv')
df5 = pd.read_csv('/content/caghifen.csv')
df6 = pd.read_csv('/content/caghin640.csv')



In [ ]:
df1.drop(['Unnamed: 0'], axis=1, inplace=True)
df2.drop(['Unnamed: 0'], axis=1, inplace=True)
df3.drop(['Unnamed: 0'], axis=1, inplace=True)
df4.drop(['Unnamed: 0'], axis=1, inplace=True)
df5.drop(['Unnamed: 0'], axis=1, inplace=True)
df6.drop(['Unnamed: 0'], axis=1, inplace=True)





In [ ]:
data = data.rename(columns = {'Sub-task A': 'label'}, inplace = False)

In [ ]:
data

,Text,label
0,Next part,0
1,Iii8mllllllm\nMdxfvb8o90lplppi0005,0
2,🤣🤣😂😂🤣🤣🤣😂osm vedio ....keep it up...make more v...,0
3,What the fuck was this? I respect shwetabh and...,0
4,Concerned authorities should bring arundathi R...,0
...,...,...
4238,Personality of a beggar lasts with her/him eve...,2
4245,Ghanta..ek no ek chutiya galti se iski video l...,2
4253,Ban suedo femists and liberals ...F..off,2
4254,Open bob and vagane,2


In [ ]:
data_traincag = pd.concat([df1,
                  df2,
                  df3,
                  df4,
                  df5,
                  df6

                 ],axis=0)

In [ ]:
 data_traincag

,Text
0,मर गीत म तइ गोर गोर i बसोड विकर बबी ओहोनकर मार...
1,लाततो न पछो अममी स पता ह कि कया व एकता क समय प...
2,मझ पता था कि व दोनो को जला म जाना चाहिए
3,फरी स कछ मागन क लिए ։
4,當 मोमिन उददीन बोकाचोडा मरी टिपपणियो को след सकत ह
...,...
442,अपने बाल बौद्ध दिवस में कुतिया बालिसों को भाग्...
443,"आप कौन हैं, अधिकारी चिलिस एनकी?आने वाले ग्रामी..."
444,असत्य और व्यर्थ
445,शालिस घर शाली फकनी


In [ ]:
df1.drop(['Unnamed: 0'], axis=1, inplace=True)
df2.drop(['Unnamed: 0'], axis=1, inplace=True)
df3.drop(['Unnamed: 0'], axis=1, inplace=True)
df4.drop(['Unnamed: 0'], axis=1, inplace=True)
df5.drop(['Unnamed: 0'], axis=1, inplace=True)





In [ ]:
data_trainoag = pd.concat([df1,
                  df2,
                  df3,
                  df4,
                  df5

                 ],axis=0)

In [ ]:
full_hin= pd.concat([data,
                    data_trainoag,
                    data_traincag,
                  
                           

                 ],axis=0)

In [ ]:
full_hin

,Text,label
0,Bollywood film dekhne ke samay logic ghar mein...,0
1,Chutiya movie...,0
2,Amrit Anand अब तो जुड़े ही है उनको बोलो जुड़ने,0
3,@Pankaj Chauhan abey tum jse aadmiyo ko ye lgt...,0
4,bro house of card ka review karona,0
...,...,...
442,अपने बाल बौद्ध दिवस में कुतिया बालिसों को भाग्...,1
443,"आप कौन हैं, अधिकारी चिलिस एनकी?आने वाले ग्रामी...",1
444,असत्य और व्यर्थ,1
445,शालिस घर शाली फकनी,1


In [ ]:
data_traincag

,Text
0,मर गीत म तइ गोर गोर i बसोड विकर बबी ओहोनकर मार...
1,लाततो न पछो अममी स पता ह कि कया व एकता क समय प...
2,मझ पता था कि व दोनो को जला म जाना चाहिए
3,फरी स कछ मागन क लिए ։
4,當 मोमिन उददीन बोकाचोडा मरी टिपपणियो को след सकत ह
...,...
442,अपने बाल बौद्ध दिवस में कुतिया बालिसों को भाग्...
443,"आप कौन हैं, अधिकारी चिलिस एनकी?आने वाले ग्रामी..."
444,असत्य और व्यर्थ
445,शालिस घर शाली फकनी


In [ ]:
data_traincag

,Text
0,feminism means equality not discrimination fir...
1,i had become ache during this movie and also p...
2,ram raja - she and a female dog - who can [UNK...
3,totally agree by ur points bro nd kabir singh....
4,one word for reasons bhaad me jaa chudail
...,...
516,"@ Itch Pong Abe Randi ki Aulad, what has happe..."
517,Arundhati your janma diya maap gyan ji hi to m...
518,mutiny
519,Kung Thu Gadhiya Hai


In [ ]:
d1 = pd.read_csv("/content/cagbeneng.csv")
d2 = pd.read_csv("/content/hinengCag.csv")
d3 = pd.read_csv("/content/hinengOag.csv")
d4 = pd.read_csv("/content/oagbeneng.csv")
d5 = pd.read_csv("/content/train.csv")



In [ ]:
d5

,ID,Text,Sub-task A,Sub-task B
0,C45.451,Next part,NAG,NGEN
1,C47.11,Iii8mllllllm\nMdxfvb8o90lplppi0005,NAG,NGEN
2,C33.79,🤣🤣😂😂🤣🤣🤣😂osm vedio ....keep it up...make more v...,NAG,NGEN
3,C4.1961,What the fuck was this? I respect shwetabh and...,NAG,NGEN
4,C10.153,Concerned authorities should bring arundathi R...,NAG,NGEN
...,...,...,...,...
4258,C10.123,Abey loudey Arnab... Did u ever see the vedios...,OAG,GEN
4259,C10.545,Arundati is very rich where she get money who ...,NAG,NGEN
4260,C4.1082,People may criticize Pratik Borade by saying t...,CAG,NGEN
4261,C33.592.4,@Naaz Sk hello,NAG,NGEN


In [ ]:
d1

,Unnamed: 0,Text
0,0,"We dont want to be caste-bound, we used to kil..."
1,1,@ Feminism is Cancer * Un Feminist Yeh Sahi Ha...
2,2,Ilia Jat Jhant Garam Hai Mukhya Jat Hu U Nehi ...
3,3,Lit Sirvans Des Bhakti Mai Bhai Item Geet Pelege
4,4,"We need men in the army, not eunuchs.Indian so..."
...,...,...
516,516,"@ Itch Pong Abe Randi ki Aulad, what has happe..."
517,517,Arundhati your janma diya maap gyan ji hi to m...
518,518,mutiny
519,519,Kung Thu Gadhiya Hai


In [ ]:
d2

,Unnamed: 0,Text,label
0,0,"We were meant to play Jaat Bandha, use Maar da...",2
1,1,@Feminism Is CANCER* Un Feminist This is true\...,2
2,2,Musk is a goth,2
3,3,Isliya Jat is a jatt...I am a Jat... you dont ...,2
4,4,Lit Sirobenko Desh Bhakti Main Bhi Item Song,2
...,...,...,...
798,798,"Make a video of some ruckus, what nonsense, th...",2
799,799,Bhadwa law doesnt work in Indian Army,2
800,800,mutiny,2
801,801,Kung Tu Gadhiya Hai,2


In [ ]:
d3

,Unnamed: 0,Text,label
0,0,There is also liberal there is also tolerant L...,1
1,1,"Does no one trust the government anymore, what...",1
2,2,"What a friend to review, write down, read a li...",1
3,3,"This happened in the right justice, which B*%$...",1
4,4,So shouldnt there be girls too,1
...,...,...,...
700,700,"We are all equal, talk u. Big countries like S...",1
701,701,Rahul Kumar Gandu This is not a law in Pakista...,1
702,702,"Well then, our population has increased from t...",1
703,703,Hear his answerYou redeemed it Pratik Bhai.......,1


In [ ]:
d4

,Unnamed: 0,Text
0,0,"@ Sucharita Tyagi Jhuthiya,"
1,1,"Liberal too, tolerant too !!! Hilarious laugh"
2,2,"What is there now on the government, not Bhosr..."
3,3,"Kya yaar review leah lee lee ko read a little,..."
4,4,"So what are you girls, no Mansi"
...,...,...
387,387,Bhai Lakhai Chaudharys video is Hatwa Diya. \ ...
388,388,Manna Awaaz Aachi Woh Lankina Kina Din Raghagi
389,389,Acha to tune is film had to be seen with dekay...
390,390,Kabir Singhs Ekke Hamad Bad article was releas...


In [ ]:
import pandas as pd
from pandas import DataFrame

In [ ]:
df = DataFrame (fNAG,columns=['Text'])

In [ ]:
df

,Text
0,Bom Bom Bajega Bom Bom
1,You will not know when a psycho stalks your si...
2,The message was very good but the laughter cam...
3,Ranga - the leaders of BJP who survived the ra...
4,Bhai Bhaut time later the street listened \nme...
...,...
1587,The Great Alexandra kept Gay in her army and r...
1588,Army Mai leaves after being a little physicall...
1589,Auto Wale Bhaiya Best
1590,Is it male or female..


In [ ]:
from google.colab import files

# e.g. save pandas output as csv
datas.to_csv('Eng_fullTranslated.csv')

# or any other file as usual
# with open('example.csv', 'w') as f:
#   f.write('your strings here')

files.download('Eng_fullTranslated.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
d = pd.read_csv("/content/ben_to_engNAG.csv")

In [ ]:
label =[]
shapna = '0'
for i in range(1592):

   label.append(shapna)

In [ ]:
from pandas import DataFrame

In [ ]:
dfOAG_label = DataFrame (label,columns=['label'])

In [ ]:
df['label'] = dfOAG_label ['label'].copy()

In [ ]:
df

,Text,label
0,Bom Bom Bajega Bom Bom,0
1,You will not know when a psycho stalks your si...,0
2,The message was very good but the laughter cam...,0
3,Ranga - the leaders of BJP who survived the ra...,0
4,Bhai Bhaut time later the street listened \nme...,0
...,...,...
1587,The Great Alexandra kept Gay in her army and r...,0
1588,Army Mai leaves after being a little physicall...,0
1589,Auto Wale Bhaiya Best,0
1590,Is it male or female..,0


In [ ]:
d1 = pd.read_csv("/content/ben_to_engNAG.csv")
d2 = pd.read_csv("/content/hin_to_engNAG.csv")
d3  = pd.read_csv("/content/eng_fulltranslated.csv")



In [ ]:
datas = pd.concat([d1,
                  d2,
                  d3

                 ],axis=0)

In [ ]:
datas

,Unnamed: 0,Text,label
0,0,😅😅😅😅😈😈😈,0.0
1,1,A glass of water dau du mori,0.0
2,2,Shotti Bhai Ranu Mandal Manush Nai,0.0
3,3,This video is the best of luck but Ranu Mandal...,0.0
4,4,Ranu Bikari Chilo Bikari R O Bikari Thakba Khanki,0.0
...,...,...,...
6679,700,"We are all equal, talk u. Big countries like S...",1.0
6680,701,Rahul Kumar Gandu This is not a law in Pakista...,1.0
6681,702,"Well then, our population has increased from t...",1.0
6682,703,Hear his answerYou redeemed it Pratik Bhai.......,1.0


In [ ]:
import pandas as pd

In [ ]:
d1 = pd.read_csv("/content/hinengOag.csv")
d2 = pd.read_csv("/content/oagbeneng.csv")

In [ ]:
d1

,Unnamed: 0,Text,label
0,0,There is also liberal there is also tolerant L...,1
1,1,"Does no one trust the government anymore, what...",1
2,2,"What a friend to review, write down, read a li...",1
3,3,"This happened in the right justice, which B*%$...",1
4,4,So shouldnt there be girls too,1
...,...,...,...
700,700,"We are all equal, talk u. Big countries like S...",1
701,701,Rahul Kumar Gandu This is not a law in Pakista...,1
702,702,"Well then, our population has increased from t...",1
703,703,Hear his answerYou redeemed it Pratik Bhai.......,1


In [ ]:
d2

,Unnamed: 0,Text
0,0,"@ Sucharita Tyagi Jhuthiya,"
1,1,"Liberal too, tolerant too !!! Hilarious laugh"
2,2,"What is there now on the government, not Bhosr..."
3,3,"Kya yaar review leah lee lee ko read a little,..."
4,4,"So what are you girls, no Mansi"
...,...,...
387,387,Bhai Lakhai Chaudharys video is Hatwa Diya. \ ...
388,388,Manna Awaaz Aachi Woh Lankina Kina Din Raghagi
389,389,Acha to tune is film had to be seen with dekay...
390,390,Kabir Singhs Ekke Hamad Bad article was releas...
